# ETL Silver → Gold Layer
## Data Warehouse Dengue 2025 - Star Schema
### Nomenclatura: DICIONARIO_MNEMONICOS.md

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

DB_CONFIG = {'host': 'localhost', 'port': 5432, 'database': 'gis', 'user': 'postgres', 'password': 'postgres'}
BATCH_SIZE = 50_000
print("✅ Setup")

In [ ]:
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM silver.dengue_notificacoes")
total_silver = cursor.fetchone()[0]
print(f"✅ Conectado | Silver: {total_silver:,} registros")

In [ ]:
query = """
SELECT id_agravo, dt_notific, sg_uf_not, nu_idade_n, tp_idade, cs_sexo, cs_raca, classi_fin, evolucao, dt_sin_pri,
       febre, mialgia, cefaleia, exantema, vomito, nausea, dor_costas, conjuntvit, artrite,
       artralgia, petequia_n, leucopenia, laco, dor_retro, diabetes, hematolog, hepatopat, renal, hospitaliz
FROM silver.dengue_notificacoes
"""
print("⏳ Carregando Silver...")
df_silver = pd.read_sql(query, conn)
print(f"✅ Carregados: {len(df_silver):,} registros")

In [ ]:
# dim_tmp (Tempo)
DIAS = {0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sábado', 6: 'Domingo'}
datas = pd.to_datetime(df_silver['dt_notific'].dropna().unique())
dim_tempo = [{'dt_completa': d.date(), 'nr_ano': d.year, 'nr_mes': d.month, 'nr_dia': d.day,
              'nr_trimestre': (d.month-1)//3+1, 'nr_semana_epi': d.isocalendar()[1],
              'nr_dia_semana': d.dayofweek+1, 'nm_dia': DIAS[d.dayofweek],
              'flag_fim_semana': d.dayofweek>=5, 'ds_mes_ano': f"{d.year}-{d.month:02d}",
              'ds_ano_trimestre': f"{d.year}-Q{(d.month-1)//3+1}"} for d in sorted(datas)]
df_dim_tempo = pd.DataFrame(dim_tempo)
print(f"✅ dim_tmp: {len(df_dim_tempo)} datas")

In [ ]:
# dim_loc (Localização)
UFS = {'AC': ('Acre','Norte',12), 'AL': ('Alagoas','Nordeste',27), 'AP': ('Amapá','Norte',16),
       'AM': ('Amazonas','Norte',13), 'BA': ('Bahia','Nordeste',29), 'CE': ('Ceará','Nordeste',23),
       'DF': ('Distrito Federal','Centro-Oeste',53), 'ES': ('Espírito Santo','Sudeste',32),
       'GO': ('Goiás','Centro-Oeste',52), 'MA': ('Maranhão','Nordeste',21), 'MT': ('Mato Grosso','Centro-Oeste',51),
       'MS': ('Mato Grosso do Sul','Centro-Oeste',50), 'MG': ('Minas Gerais','Sudeste',31),
       'PA': ('Pará','Norte',15), 'PB': ('Paraíba','Nordeste',25), 'PR': ('Paraná','Sul',41),
       'PE': ('Pernambuco','Nordeste',26), 'PI': ('Piauí','Nordeste',22), 'RJ': ('Rio de Janeiro','Sudeste',33),
       'RN': ('Rio Grande do Norte','Nordeste',24), 'RS': ('Rio Grande do Sul','Sul',43),
       'RO': ('Rondônia','Norte',11), 'RR': ('Roraima','Norte',14), 'SC': ('Santa Catarina','Sul',42),
       'SP': ('São Paulo','Sudeste',35), 'SE': ('Sergipe','Nordeste',28), 'TO': ('Tocantins','Norte',17)}
ufs_dados = df_silver['sg_uf_not'].dropna().unique()
dim_loc = [{'sg_uf': uf, 'nm_uf': UFS[uf][0], 'nm_regiao': UFS[uf][1], 'cd_ibge': UFS[uf][2], 'nm_capital': 'N/A'}
           for uf in sorted(ufs_dados) if uf in UFS]
df_dim_loc = pd.DataFrame(dim_loc)
print(f"✅ dim_loc: {len(df_dim_loc)} UFs")

In [ ]:
# dim_pac (Paciente)
def calc_idade(nu, tp): 
    if pd.isna(nu) or pd.isna(tp): return None
    t = int(tp)
    return float(nu) if t==4 else float(nu)/12 if t==3 else float(nu)/365 if t==2 else float(nu)/8760

def faixa_et(i):
    if pd.isna(i): return 'UNKNOWN'
    if i<1: return '<1 ano'
    for lim, desc in [(5,'1-4'),(10,'5-9'),(15,'10-14'),(20,'15-19'),(30,'20-29'),(40,'30-39'),(50,'40-49'),(60,'50-59'),(70,'60-69'),(80,'70-79')]:
        if i<lim: return f'{desc} anos'
    return '80+ anos'

SEXO = {'M': 'Masculino', 'F': 'Feminino', 'I': 'Ignorado'}
RACA = {'1': 'Branca', '2': 'Preta', '3': 'Amarela', '4': 'Parda', '5': 'Indígena', '9': 'Ignorado'}
FAIXA_DET = lambda i: 'UNKNOWN' if pd.isna(i) else 'Lactente' if i<1 else 'Criança' if i<12 else 'Adolescente' if i<18 else 'Adulto' if i<60 else 'Idoso'

df_silver['idade_anos'] = df_silver.apply(lambda x: calc_idade(x['nu_idade_n'], x['tp_idade']), axis=1)
df_pac = df_silver[['idade_anos', 'cs_sexo', 'cs_raca']].copy()
df_pac['ds_faixa_etaria'] = df_pac['idade_anos'].apply(faixa_et)
df_pac['ds_sexo'] = df_pac['cs_sexo'].map(SEXO).fillna('UNKNOWN')
df_pac['ds_raca'] = df_pac['cs_raca'].astype(str).map(RACA).fillna('UNKNOWN')
df_pac['ds_faixa_etaria_det'] = df_pac['idade_anos'].apply(FAIXA_DET)
df_pac['nk_demografica'] = df_pac['ds_faixa_etaria']+'|'+df_pac['ds_sexo']+'|'+df_pac['ds_raca']
df_dim_pac = df_pac[['nk_demografica','ds_faixa_etaria','ds_sexo','ds_raca','ds_faixa_etaria_det']].drop_duplicates()
print(f"✅ dim_pac: {len(df_dim_pac)} perfis")

In [ ]:
# dim_cls (Classificação)
CLASSIF = {'5': ('Descartado','Negativo','N/A',None,False), '10': ('Dengue','Confirmado','Leve','A90',True),
           '11': ('Dengue com Alarme','Confirmado','Moderado','A91',True), '12': ('Dengue Grave','Confirmado','Grave','A91',True),
           '13': ('Chikungunya','Confirmado','Variável','A92.0',True), '8': ('Inconclusivo','Indeterminado','N/A',None,False)}
classifs = df_silver['classi_fin'].dropna().unique()
dim_cls = []
for c in sorted(classifs):
    cs = str(int(c))
    if cs in CLASSIF:
        i = CLASSIF[cs]
        dim_cls.append({'cd_classificacao': cs, 'ds_classificacao': i[0], 'ds_grupo': i[1], 'ds_gravidade': i[2], 'cd_cid': i[3], 'flag_confirmado': i[4]})
df_dim_cls = pd.DataFrame(dim_cls)
print(f"✅ dim_cls: {len(df_dim_cls)} tipos")

In [ ]:
# dim_evl (Evolução)
EVOL = {'1': ('Cura','Favorável',False,'Baixa'), '2': ('Óbito pelo agravo','Óbito',True,'Crítica'),
        '3': ('Óbito outras causas','Óbito',True,'Crítica'), '4': ('Óbito em investigação','Óbito',True,'Crítica'),
        '9': ('Ignorado','Indeterminado',False,'Indeterminada')}
evolucoes = df_silver['evolucao'].dropna().unique()
dim_evl = []
for e in sorted(evolucoes):
    es = str(int(e))
    if es in EVOL:
        i = EVOL[es]
        dim_evl.append({'cd_evolucao': es, 'ds_evolucao': i[0], 'ds_tipo_evolucao': i[1], 'flag_obito': i[2], 'ds_gravidade_desfecho': i[3]})
df_dim_evl = pd.DataFrame(dim_evl)
print(f"✅ dim_evl: {len(df_dim_evl)} tipos")

In [ ]:
# dim_snt (Sintomas)
SINT_COLS = ['febre','mialgia','cefaleia','exantema','vomito','nausea','dor_costas','conjuntvit','artrite']
ALARM_COLS = ['artralgia','petequia_n','leucopenia','laco','dor_retro','diabetes','hematolog','hepatopat']
df_silver['qtd_sint'] = df_silver[SINT_COLS].apply(lambda x: (x==1).sum(), axis=1)
df_silver['qtd_alarm'] = df_silver[ALARM_COLS].apply(lambda x: (x==1).sum(), axis=1)
faixa_s = lambda q: 'Nenhum' if q==0 else 'Poucos (1-2)' if q<=2 else 'Moderado (3-5)' if q<=5 else 'Muitos (6+)'
faixa_a = lambda q: 'Nenhum' if q==0 else 'Poucos (1-2)' if q<=2 else 'Múltiplos (3+)'
perfil = lambda s,a: 'Assintomático' if s==0 and a==0 else 'Dengue Clássica' if a==0 else 'Dengue com Alarme' if a<=2 else 'Dengue Grave'
df_snt = df_silver[['qtd_sint','qtd_alarm']].copy()
df_snt['ds_faixa_sintomas'] = df_snt['qtd_sint'].apply(faixa_s)
df_snt['ds_faixa_alarmes'] = df_snt['qtd_alarm'].apply(faixa_a)
df_snt['ds_perfil_clinico'] = df_snt.apply(lambda x: perfil(x['qtd_sint'],x['qtd_alarm']), axis=1)
df_snt['flag_tem_sintomas'] = df_snt['qtd_sint']>0
df_snt['flag_tem_alarmes'] = df_snt['qtd_alarm']>0
df_snt['nk_sintomas'] = df_snt['ds_faixa_sintomas']+'|'+df_snt['ds_faixa_alarmes']
df_dim_snt = df_snt[['nk_sintomas','ds_faixa_sintomas','ds_faixa_alarmes','ds_perfil_clinico','flag_tem_sintomas','flag_tem_alarmes']].drop_duplicates()
print(f"✅ dim_snt: {len(df_dim_snt)} perfis")

In [ ]:
# CARGA DAS DIMENSÕES
def load_dim(df, table, cols):
    sql = f"INSERT INTO {table} ({','.join(cols)}) VALUES ({','.join(['%s']*len(cols))})"
    data = [tuple(r[c] for c in cols) for _,r in df.iterrows()]
    cursor.executemany(sql, data)
    conn.commit()
    return len(data)

print("⏳ Carregando dimensões...")
n = load_dim(df_dim_tempo, 'gold.dim_tmp', ['dt_completa','nr_ano','nr_mes','nr_dia','nr_trimestre','nr_semana_epi','nr_dia_semana','nm_dia','flag_fim_semana','ds_mes_ano','ds_ano_trimestre'])
print(f"   dim_tmp: {n}")
n = load_dim(df_dim_loc, 'gold.dim_loc', ['sg_uf','nm_uf','nm_regiao','cd_ibge','nm_capital'])
print(f"   dim_loc: {n}")
n = load_dim(df_dim_pac, 'gold.dim_pac', ['nk_demografica','ds_faixa_etaria','ds_sexo','ds_raca','ds_faixa_etaria_det'])
print(f"   dim_pac: {n}")
n = load_dim(df_dim_cls, 'gold.dim_cls', ['cd_classificacao','ds_classificacao','ds_grupo','ds_gravidade','cd_cid','flag_confirmado'])
print(f"   dim_cls: {n}")
n = load_dim(df_dim_evl, 'gold.dim_evl', ['cd_evolucao','ds_evolucao','ds_tipo_evolucao','flag_obito','ds_gravidade_desfecho'])
print(f"   dim_evl: {n}")
n = load_dim(df_dim_snt, 'gold.dim_snt', ['nk_sintomas','ds_faixa_sintomas','ds_faixa_alarmes','ds_perfil_clinico','flag_tem_sintomas','flag_tem_alarmes'])
print(f"   dim_snt: {n}")
print("✅ Dimensões carregadas")

In [ ]:
# LOOKUPS
print("⏳ Criando lookups...")
cursor.execute("SELECT sk_tmp, dt_completa FROM gold.dim_tmp")
lk_tmp = dict(cursor.fetchall()[::-1])  # dt_completa -> sk_tmp
lk_tmp = {v:k for k,v in dict(cursor.fetchall()).items()}  # inverter
cursor.execute("SELECT sk_tmp, dt_completa FROM gold.dim_tmp")
lk_tmp = {r[1]:r[0] for r in cursor.fetchall()}

cursor.execute("SELECT sk_loc, sg_uf FROM gold.dim_loc")
lk_loc = {r[1]:r[0] for r in cursor.fetchall()}

cursor.execute("SELECT sk_pac, nk_demografica FROM gold.dim_pac")
lk_pac = {r[1]:r[0] for r in cursor.fetchall()}

cursor.execute("SELECT sk_cls, cd_classificacao FROM gold.dim_cls")
lk_cls = {r[1]:r[0] for r in cursor.fetchall()}

cursor.execute("SELECT sk_evl, cd_evolucao FROM gold.dim_evl")
lk_evl = {r[1]:r[0] for r in cursor.fetchall()}

cursor.execute("SELECT sk_snt, nk_sintomas FROM gold.dim_snt")
lk_snt = {r[1]:r[0] for r in cursor.fetchall()}

print(f"✅ Lookups: tmp={len(lk_tmp)}, loc={len(lk_loc)}, pac={len(lk_pac)}, cls={len(lk_cls)}, evl={len(lk_evl)}, snt={len(lk_snt)}")

In [ ]:
# PREPARAR FATO
print("⏳ Preparando tabela fato...")
df_silver['nk_demografica'] = df_pac['nk_demografica']
df_silver['nk_sintomas'] = df_snt['nk_sintomas']

df_fato = pd.DataFrame()
df_fato['nk_notificacao'] = df_silver['id_agravo']
df_fato['fk_tmp'] = df_silver['dt_notific'].apply(lambda x: lk_tmp.get(pd.to_datetime(x).date(),-1) if pd.notna(x) else -1)
df_fato['fk_loc'] = df_silver['sg_uf_not'].apply(lambda x: lk_loc.get(x,-1) if pd.notna(x) else -1)
df_fato['fk_pac'] = df_silver['nk_demografica'].apply(lambda x: lk_pac.get(x,-1) if pd.notna(x) else -1)
df_fato['fk_cls'] = df_silver['classi_fin'].apply(lambda x: lk_cls.get(str(int(x)),-1) if pd.notna(x) else -1)
df_fato['fk_evl'] = df_silver['evolucao'].apply(lambda x: lk_evl.get(str(int(x)),-1) if pd.notna(x) else -1)
df_fato['fk_snt'] = df_silver['nk_sintomas'].apply(lambda x: lk_snt.get(x,-1) if pd.notna(x) else -1)
df_fato['vl_confirmado'] = df_silver['classi_fin'].apply(lambda x: 1 if pd.notna(x) and int(x) in [10,11,12,13] else 0)
df_fato['vl_grave'] = df_silver['classi_fin'].apply(lambda x: 1 if pd.notna(x) and int(x)==12 else 0)
df_fato['vl_obito'] = df_silver['evolucao'].apply(lambda x: 1 if pd.notna(x) and int(x) in [2,3,4] else 0)
df_fato['vl_hospitalizado'] = df_silver['hospitaliz'].apply(lambda x: 1 if pd.notna(x) and int(x)==1 else 0)
df_fato['vl_qtd_sintomas'] = df_silver['qtd_sint'].fillna(0).astype(int)
df_fato['vl_qtd_alarmes'] = df_silver['qtd_alarm'].fillna(0).astype(int)
df_fato['vl_idade_anos'] = df_silver['idade_anos']
df_fato['dt_notificacao'] = pd.to_datetime(df_silver['dt_notific']).dt.date
df_fato['dt_sintomas'] = pd.to_datetime(df_silver['dt_sin_pri']).dt.date
print(f"✅ Fato preparada: {len(df_fato):,} registros")

In [ ]:
# CARGA FATO EM BATCHES
cols = ['nk_notificacao','fk_tmp','fk_loc','fk_pac','fk_cls','fk_evl','fk_snt',
        'vl_confirmado','vl_grave','vl_obito','vl_hospitalizado','vl_qtd_sintomas','vl_qtd_alarmes','vl_idade_anos',
        'dt_notificacao','dt_sintomas']
sql = f"INSERT INTO gold.ft_deng ({','.join(cols)}) VALUES %s"
total = len(df_fato)
batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
print(f"⏳ Carregando {total:,} registros em {batches} batches...")

for i in range(batches):
    start = i * BATCH_SIZE
    end = min((i+1)*BATCH_SIZE, total)
    batch = df_fato.iloc[start:end]
    data = [tuple(r[c] if pd.notna(r[c]) else None for c in cols) for _,r in batch.iterrows()]
    execute_values(cursor, sql, data)
    conn.commit()
    print(f"   Batch {i+1}/{batches}: {end:,} ({(i+1)/batches*100:.0f}%)")
    
print(f"\n✅ Carga concluída: {total:,} registros")

In [ ]:
# VALIDAÇÃO FINAL
print("🔍 VALIDAÇÃO FINAL")
cursor.execute("SELECT COUNT(*) FROM gold.ft_deng")
gold = cursor.fetchone()[0]
print(f"\n📊 VOLUMETRIA: Silver={total_silver:,} | Gold={gold:,} | {'✅ OK' if gold==total_silver else '❌ DIFF'}")

cursor.execute("SELECT SUM(vl_confirmado), SUM(vl_grave), SUM(vl_obito) FROM gold.ft_deng")
m = cursor.fetchone()
print(f"\n📈 MÉTRICAS: Confirmados={m[0]:,} | Graves={m[1]:,} | Óbitos={m[2]:,}")
if m[0] > 0:
    print(f"   Taxa letalidade: {m[2]/m[0]*100:.4f}%")

cursor.close()
conn.close()
print("\n✅ ETL SILVER → GOLD CONCLUÍDO COM SUCESSO!")

# ETL Silver → Gold Layer
## Data Warehouse Dengue 2025 - Star Schema

### Objetivo
Transformar dados normalizados da Silver Layer em um Star Schema otimizado para análises OLAP.

### Nomenclatura Padronizada (DICIONARIO_MNEMONICOS.md)
- **Tabelas**: dim_tmp, dim_loc, dim_pac, dim_cls, dim_evl, dim_snt, ft_deng
- **Prefixos**: sk_ (surrogate key), fk_ (foreign key), nk_ (natural key), vl_ (valor), dt_ (data), nr_ (número), ds_ (descrição), nm_ (nome), sg_ (sigla), cd_ (código), flag_ (booleano), ts_ (timestamp)

## 1. Setup e Configuração

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

# Configuração do banco de dados
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'gis',
    'user': 'postgres',
    'password': 'postgres'
}

# Constantes
VOLUMETRIA_ESPERADA = 1_661_634
BATCH_SIZE = 50_000

print("✅ Setup concluído")

## 2. Conexão e Verificação Silver Layer

In [ ]:
# Conectar ao banco
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()

# Verificar dados na Silver
query_silver = """
SELECT COUNT(*) as total,
       MIN(dt_notific) as data_min,
       MAX(dt_notific) as data_max,
       COUNT(DISTINCT sg_uf_not) as qtd_ufs
FROM silver.dengue_notificacoes
"""
cursor.execute(query_silver)
result = cursor.fetchone()

print("📊 Silver Layer - Resumo:")
print(f"   Total registros: {result[0]:,}")
print(f"   Período: {result[1]} a {result[2]}")
print(f"   UFs: {result[3]}")
print(f"\n✅ Volumetria esperada: {VOLUMETRIA_ESPERADA:,}")
print(f"   Status: {'✅ OK' if result[0] == VOLUMETRIA_ESPERADA else '⚠️ DIFERENÇA'}")

## 3. Carregar Dados Silver em Memória

In [ ]:
# Query otimizada para ETL
query = """
SELECT 
    id_agravo,
    dt_notific,
    sg_uf_not,
    nu_idade_n,
    tp_idade,
    cs_sexo,
    cs_raca,
    classi_fin,
    evolucao,
    dt_sin_pri,
    febre,
    mialgia,
    cefaleia,
    exantema,
    vomito,
    nausea,
    dor_costas,
    conjuntvit,
    artrite,
    artralgia,
    petequia_n,
    leucopenia,
    laco,
    dor_retro,
    diabetes,
    hematolog,
    hepatopat,
    renal,
    hipertensa,
    acido_pept,
    auto_imune,
    hospitaliz
FROM silver.dengue_notificacoes
"""

print("⏳ Carregando dados Silver para memória...")
df_silver = pd.read_sql(query, conn)
print(f"✅ Carregados {len(df_silver):,} registros")
print(f"   Memória: {df_silver.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# Verificar completude dos campos críticos
campos_criticos = ['dt_notific', 'sg_uf_not', 'classi_fin', 'evolucao']
print("📋 Completude campos críticos:")
for col in campos_criticos:
    pct = (1 - df_silver[col].isna().mean()) * 100
    print(f"   {col}: {pct:.1f}%")

## 4. Construção das Dimensões
### 4.1 dim_tmp (Dimensão Tempo)

In [ ]:
# Dicionário de dias da semana em português
DIAS_SEMANA = {0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sábado', 6: 'Domingo'}

# Extrair datas únicas
datas_unicas = pd.to_datetime(df_silver['dt_notific'].dropna().unique())
print(f"📅 Datas únicas para dim_tmp: {len(datas_unicas)}")

# Construir dados da dimensão
dim_tempo_data = []
for d in sorted(datas_unicas):
    data = d.date()
    dia_semana_python = d.dayofweek
    
    dim_tempo_data.append({
        'dt_completa': data,
        'nr_ano': d.year,
        'nr_mes': d.month,
        'nr_dia': d.day,
        'nr_trimestre': (d.month - 1) // 3 + 1,
        'nr_semana_epi': d.isocalendar()[1],
        'nr_dia_semana': dia_semana_python + 1,
        'nm_dia': DIAS_SEMANA[dia_semana_python],
        'flag_fim_semana': dia_semana_python >= 5,
        'ds_mes_ano': f"{d.year}-{d.month:02d}",
        'ds_ano_trimestre': f"{d.year}-Q{(d.month - 1) // 3 + 1}"
    })

df_dim_tempo = pd.DataFrame(dim_tempo_data)
print(f"✅ dim_tmp preparada: {len(df_dim_tempo)} registros")
df_dim_tempo.head()

### 4.2 dim_loc (Dimensão Localização)

In [ ]:
# Mapeamento UFs do Brasil com metadados
UFS_BRASIL = {
    'AC': {'nm_uf': 'Acre', 'nm_regiao': 'Norte', 'cd_ibge': 12, 'nm_capital': 'Rio Branco'},
    'AL': {'nm_uf': 'Alagoas', 'nm_regiao': 'Nordeste', 'cd_ibge': 27, 'nm_capital': 'Maceió'},
    'AP': {'nm_uf': 'Amapá', 'nm_regiao': 'Norte', 'cd_ibge': 16, 'nm_capital': 'Macapá'},
    'AM': {'nm_uf': 'Amazonas', 'nm_regiao': 'Norte', 'cd_ibge': 13, 'nm_capital': 'Manaus'},
    'BA': {'nm_uf': 'Bahia', 'nm_regiao': 'Nordeste', 'cd_ibge': 29, 'nm_capital': 'Salvador'},
    'CE': {'nm_uf': 'Ceará', 'nm_regiao': 'Nordeste', 'cd_ibge': 23, 'nm_capital': 'Fortaleza'},
    'DF': {'nm_uf': 'Distrito Federal', 'nm_regiao': 'Centro-Oeste', 'cd_ibge': 53, 'nm_capital': 'Brasília'},
    'ES': {'nm_uf': 'Espírito Santo', 'nm_regiao': 'Sudeste', 'cd_ibge': 32, 'nm_capital': 'Vitória'},
    'GO': {'nm_uf': 'Goiás', 'nm_regiao': 'Centro-Oeste', 'cd_ibge': 52, 'nm_capital': 'Goiânia'},
    'MA': {'nm_uf': 'Maranhão', 'nm_regiao': 'Nordeste', 'cd_ibge': 21, 'nm_capital': 'São Luís'},
    'MT': {'nm_uf': 'Mato Grosso', 'nm_regiao': 'Centro-Oeste', 'cd_ibge': 51, 'nm_capital': 'Cuiabá'},
    'MS': {'nm_uf': 'Mato Grosso do Sul', 'nm_regiao': 'Centro-Oeste', 'cd_ibge': 50, 'nm_capital': 'Campo Grande'},
    'MG': {'nm_uf': 'Minas Gerais', 'nm_regiao': 'Sudeste', 'cd_ibge': 31, 'nm_capital': 'Belo Horizonte'},
    'PA': {'nm_uf': 'Pará', 'nm_regiao': 'Norte', 'cd_ibge': 15, 'nm_capital': 'Belém'},
    'PB': {'nm_uf': 'Paraíba', 'nm_regiao': 'Nordeste', 'cd_ibge': 25, 'nm_capital': 'João Pessoa'},
    'PR': {'nm_uf': 'Paraná', 'nm_regiao': 'Sul', 'cd_ibge': 41, 'nm_capital': 'Curitiba'},
    'PE': {'nm_uf': 'Pernambuco', 'nm_regiao': 'Nordeste', 'cd_ibge': 26, 'nm_capital': 'Recife'},
    'PI': {'nm_uf': 'Piauí', 'nm_regiao': 'Nordeste', 'cd_ibge': 22, 'nm_capital': 'Teresina'},
    'RJ': {'nm_uf': 'Rio de Janeiro', 'nm_regiao': 'Sudeste', 'cd_ibge': 33, 'nm_capital': 'Rio de Janeiro'},
    'RN': {'nm_uf': 'Rio Grande do Norte', 'nm_regiao': 'Nordeste', 'cd_ibge': 24, 'nm_capital': 'Natal'},
    'RS': {'nm_uf': 'Rio Grande do Sul', 'nm_regiao': 'Sul', 'cd_ibge': 43, 'nm_capital': 'Porto Alegre'},
    'RO': {'nm_uf': 'Rondônia', 'nm_regiao': 'Norte', 'cd_ibge': 11, 'nm_capital': 'Porto Velho'},
    'RR': {'nm_uf': 'Roraima', 'nm_regiao': 'Norte', 'cd_ibge': 14, 'nm_capital': 'Boa Vista'},
    'SC': {'nm_uf': 'Santa Catarina', 'nm_regiao': 'Sul', 'cd_ibge': 42, 'nm_capital': 'Florianópolis'},
    'SP': {'nm_uf': 'São Paulo', 'nm_regiao': 'Sudeste', 'cd_ibge': 35, 'nm_capital': 'São Paulo'},
    'SE': {'nm_uf': 'Sergipe', 'nm_regiao': 'Nordeste', 'cd_ibge': 28, 'nm_capital': 'Aracaju'},
    'TO': {'nm_uf': 'Tocantins', 'nm_regiao': 'Norte', 'cd_ibge': 17, 'nm_capital': 'Palmas'}
}

# Verificar UFs presentes na Silver
ufs_silver = df_silver['sg_uf_not'].dropna().unique()
print(f"📍 UFs na Silver: {len(ufs_silver)}")
print(f"   UFs: {sorted(ufs_silver)}")

# Construir dimensão apenas com UFs presentes
dim_localizacao_data = []
for uf in sorted(ufs_silver):
    if uf in UFS_BRASIL:
        info = UFS_BRASIL[uf]
        dim_localizacao_data.append({
            'sg_uf': uf,
            'nm_uf': info['nm_uf'],
            'nm_regiao': info['nm_regiao'],
            'cd_ibge': info['cd_ibge'],
            'nm_capital': info['nm_capital']
        })

df_dim_localizacao = pd.DataFrame(dim_localizacao_data)
print(f"✅ dim_loc preparada: {len(df_dim_localizacao)} registros")
df_dim_localizacao.head()

### 4.3 dim_pac (Dimensão Paciente)

In [ ]:
def calcular_idade_anos(nu_idade, tp_idade):
    """Converte idade para anos baseado no tipo (1=hora, 2=dia, 3=mês, 4=ano)"""
    if pd.isna(nu_idade) or pd.isna(tp_idade):
        return None
    tp = int(tp_idade)
    idade = float(nu_idade)
    if tp == 4:  # Anos
        return idade
    elif tp == 3:  # Meses
        return idade / 12
    elif tp == 2:  # Dias
        return idade / 365
    elif tp == 1:  # Horas
        return idade / 8760
    return None

def classificar_faixa_etaria(idade_anos):
    """Classifica em faixas etárias epidemiológicas"""
    if pd.isna(idade_anos):
        return 'UNKNOWN'
    if idade_anos < 1:
        return '< 1 ano'
    elif idade_anos < 5:
        return '1-4 anos'
    elif idade_anos < 10:
        return '5-9 anos'
    elif idade_anos < 15:
        return '10-14 anos'
    elif idade_anos < 20:
        return '15-19 anos'
    elif idade_anos < 30:
        return '20-29 anos'
    elif idade_anos < 40:
        return '30-39 anos'
    elif idade_anos < 50:
        return '40-49 anos'
    elif idade_anos < 60:
        return '50-59 anos'
    elif idade_anos < 70:
        return '60-69 anos'
    elif idade_anos < 80:
        return '70-79 anos'
    else:
        return '80+ anos'

def classificar_faixa_detalhada(idade_anos):
    """Faixas mais granulares para análises específicas"""
    if pd.isna(idade_anos):
        return 'UNKNOWN'
    if idade_anos < 1:
        return 'Lactente'
    elif idade_anos < 12:
        return 'Criança'
    elif idade_anos < 18:
        return 'Adolescente'
    elif idade_anos < 60:
        return 'Adulto'
    else:
        return 'Idoso'

# Mapeamentos
SEXO_MAP = {'M': 'Masculino', 'F': 'Feminino', 'I': 'Ignorado'}
RACA_MAP = {
    '1': 'Branca', '2': 'Preta', '3': 'Amarela',
    '4': 'Parda', '5': 'Indígena', '9': 'Ignorado'
}

# Calcular idade em anos
df_silver['idade_anos'] = df_silver.apply(
    lambda x: calcular_idade_anos(x['nu_idade_n'], x['tp_idade']), axis=1
)

# Criar combinações únicas de perfil demográfico
df_paciente_unique = df_silver[['idade_anos', 'cs_sexo', 'cs_raca']].copy()
df_paciente_unique['ds_faixa_etaria'] = df_paciente_unique['idade_anos'].apply(classificar_faixa_etaria)
df_paciente_unique['ds_sexo'] = df_paciente_unique['cs_sexo'].map(SEXO_MAP).fillna('UNKNOWN')
df_paciente_unique['ds_raca'] = df_paciente_unique['cs_raca'].astype(str).map(RACA_MAP).fillna('UNKNOWN')
df_paciente_unique['ds_faixa_etaria_det'] = df_paciente_unique['idade_anos'].apply(classificar_faixa_detalhada)
df_paciente_unique['nk_demografica'] = (df_paciente_unique['ds_faixa_etaria'] + '|' + 
                                         df_paciente_unique['ds_sexo'] + '|' + 
                                         df_paciente_unique['ds_raca'])

# Dimensão única
df_dim_paciente = df_paciente_unique[['nk_demografica', 'ds_faixa_etaria', 'ds_sexo', 
                                       'ds_raca', 'ds_faixa_etaria_det']].drop_duplicates()

print(f"✅ dim_pac preparada: {len(df_dim_paciente)} combinações únicas")
df_dim_paciente.head()

### 4.4 dim_cls (Dimensão Classificação)

In [ ]:
# Mapeamento de classificações SINAN
CLASSIF_MAP = {
    '5': {'ds_classificacao': 'Descartado', 'ds_grupo': 'Negativo', 'ds_gravidade': 'N/A', 'cd_cid': None, 'flag_confirmado': False},
    '10': {'ds_classificacao': 'Dengue', 'ds_grupo': 'Confirmado', 'ds_gravidade': 'Leve', 'cd_cid': 'A90', 'flag_confirmado': True},
    '11': {'ds_classificacao': 'Dengue com Sinais de Alarme', 'ds_grupo': 'Confirmado', 'ds_gravidade': 'Moderado', 'cd_cid': 'A91', 'flag_confirmado': True},
    '12': {'ds_classificacao': 'Dengue Grave', 'ds_grupo': 'Confirmado', 'ds_gravidade': 'Grave', 'cd_cid': 'A91', 'flag_confirmado': True},
    '13': {'ds_classificacao': 'Chikungunya', 'ds_grupo': 'Confirmado', 'ds_gravidade': 'Variável', 'cd_cid': 'A92.0', 'flag_confirmado': True},
    '8': {'ds_classificacao': 'Inconclusivo', 'ds_grupo': 'Indeterminado', 'ds_gravidade': 'N/A', 'cd_cid': None, 'flag_confirmado': False}
}

# Identificar classificações presentes
classificacoes = df_silver['classi_fin'].dropna().unique()
print(f"📋 Classificações na Silver: {sorted(classificacoes)}")

# Construir dimensão
dim_classificacao_data = []
for classif in sorted(classificacoes):
    classif_str = str(int(classif)) if pd.notna(classif) else 'UNKNOWN'
    if classif_str in CLASSIF_MAP:
        info = CLASSIF_MAP[classif_str]
        dim_classificacao_data.append({
            'cd_classificacao': classif_str,
            'ds_classificacao': info['ds_classificacao'],
            'ds_grupo': info['ds_grupo'],
            'ds_gravidade': info['ds_gravidade'],
            'cd_cid': info['cd_cid'],
            'flag_confirmado': info['flag_confirmado']
        })
    else:
        dim_classificacao_data.append({
            'cd_classificacao': classif_str,
            'ds_classificacao': f'Código {classif_str}',
            'ds_grupo': 'Outros',
            'ds_gravidade': 'N/A',
            'cd_cid': None,
            'flag_confirmado': False
        })

df_dim_classificacao = pd.DataFrame(dim_classificacao_data)
print(f"✅ dim_cls preparada: {len(df_dim_classificacao)} registros")
df_dim_classificacao

### 4.5 dim_evl (Dimensão Evolução)

In [ ]:
# Mapeamento de evoluções SINAN
EVOLUCAO_MAP = {
    '1': {'ds_evolucao': 'Cura', 'ds_tipo_evolucao': 'Favorável', 'flag_obito': False, 'ds_gravidade_desfecho': 'Baixa'},
    '2': {'ds_evolucao': 'Óbito pelo agravo notificado', 'ds_tipo_evolucao': 'Óbito', 'flag_obito': True, 'ds_gravidade_desfecho': 'Crítica'},
    '3': {'ds_evolucao': 'Óbito por outras causas', 'ds_tipo_evolucao': 'Óbito', 'flag_obito': True, 'ds_gravidade_desfecho': 'Crítica'},
    '4': {'ds_evolucao': 'Óbito em investigação', 'ds_tipo_evolucao': 'Óbito', 'flag_obito': True, 'ds_gravidade_desfecho': 'Crítica'},
    '9': {'ds_evolucao': 'Ignorado', 'ds_tipo_evolucao': 'Indeterminado', 'flag_obito': False, 'ds_gravidade_desfecho': 'Indeterminada'}
}

# Identificar evoluções presentes
evolucoes = df_silver['evolucao'].dropna().unique()
print(f"📈 Evoluções na Silver: {sorted(evolucoes)}")

# Construir dimensão
dim_evolucao_data = []
for evol in sorted(evolucoes):
    evol_str = str(int(evol)) if pd.notna(evol) else 'UNKNOWN'
    if evol_str in EVOLUCAO_MAP:
        info = EVOLUCAO_MAP[evol_str]
        dim_evolucao_data.append({
            'cd_evolucao': evol_str,
            'ds_evolucao': info['ds_evolucao'],
            'ds_tipo_evolucao': info['ds_tipo_evolucao'],
            'flag_obito': info['flag_obito'],
            'ds_gravidade_desfecho': info['ds_gravidade_desfecho']
        })
    else:
        dim_evolucao_data.append({
            'cd_evolucao': evol_str,
            'ds_evolucao': f'Código {evol_str}',
            'ds_tipo_evolucao': 'Outros',
            'flag_obito': False,
            'ds_gravidade_desfecho': 'Indeterminada'
        })

df_dim_evolucao = pd.DataFrame(dim_evolucao_data)
print(f"✅ dim_evl preparada: {len(df_dim_evolucao)} registros")
df_dim_evolucao

### 4.6 dim_snt (Dimensão Sintomas - Agregada)

In [ ]:
# Colunas de sintomas e alarmes
SINTOMAS_COLS = ['febre', 'mialgia', 'cefaleia', 'exantema', 'vomito', 'nausea', 'dor_costas', 'conjuntvit', 'artrite']
ALARMES_COLS = ['artralgia', 'petequia_n', 'leucopenia', 'laco', 'dor_retro', 'diabetes', 'hematolog', 'hepatopat']

def classificar_faixa_sintomas(qtd):
    if qtd == 0:
        return 'Nenhum'
    elif qtd <= 2:
        return 'Poucos (1-2)'
    elif qtd <= 5:
        return 'Moderado (3-5)'
    else:
        return 'Muitos (6+)'

def classificar_faixa_alarmes(qtd):
    if qtd == 0:
        return 'Nenhum'
    elif qtd <= 2:
        return 'Poucos (1-2)'
    else:
        return 'Múltiplos (3+)'

def classificar_perfil_clinico(qtd_sint, qtd_alarm):
    if qtd_sint == 0 and qtd_alarm == 0:
        return 'Assintomático'
    elif qtd_alarm == 0:
        return 'Dengue Clássica'
    elif qtd_alarm <= 2:
        return 'Dengue com Alarme'
    else:
        return 'Dengue Grave'

# Calcular quantidades
df_silver['qtd_sintomas'] = df_silver[SINTOMAS_COLS].apply(
    lambda x: (x == 1).sum(), axis=1
)
df_silver['qtd_alarmes'] = df_silver[ALARMES_COLS].apply(
    lambda x: (x == 1).sum(), axis=1
)

# Criar perfis únicos
df_sintomas_unique = df_silver[['qtd_sintomas', 'qtd_alarmes']].copy()
df_sintomas_unique['ds_faixa_sintomas'] = df_sintomas_unique['qtd_sintomas'].apply(classificar_faixa_sintomas)
df_sintomas_unique['ds_faixa_alarmes'] = df_sintomas_unique['qtd_alarmes'].apply(classificar_faixa_alarmes)
df_sintomas_unique['ds_perfil_clinico'] = df_sintomas_unique.apply(
    lambda x: classificar_perfil_clinico(x['qtd_sintomas'], x['qtd_alarmes']), axis=1
)
df_sintomas_unique['flag_tem_sintomas'] = df_sintomas_unique['qtd_sintomas'] > 0
df_sintomas_unique['flag_tem_alarmes'] = df_sintomas_unique['qtd_alarmes'] > 0
df_sintomas_unique['nk_sintomas'] = (df_sintomas_unique['ds_faixa_sintomas'] + '|' + 
                                      df_sintomas_unique['ds_faixa_alarmes'])

# Dimensão única
df_dim_sintomas = df_sintomas_unique[['nk_sintomas', 'ds_faixa_sintomas', 'ds_faixa_alarmes', 
                                       'ds_perfil_clinico', 'flag_tem_sintomas', 'flag_tem_alarmes']].drop_duplicates()

print(f"✅ dim_snt preparada: {len(df_dim_sintomas)} combinações únicas")
df_dim_sintomas

## 5. Carga das Dimensões no Banco

In [ ]:
def load_dimension(df, table_name, columns, cursor, conn):
    """Função genérica para carregar dimensões"""
    cols_str = ', '.join(columns)
    placeholders = ', '.join(['%s'] * len(columns))
    insert_sql = f"INSERT INTO {table_name} ({cols_str}) VALUES ({placeholders})"
    
    data = [tuple(row[col] for col in columns) for _, row in df.iterrows()]
    cursor.executemany(insert_sql, data)
    conn.commit()
    
    return len(data)

print("⏳ Carregando dimensões...")

# dim_tmp
cols_tempo = ['dt_completa', 'nr_ano', 'nr_mes', 'nr_dia', 'nr_trimestre', 'nr_semana_epi', 
              'nr_dia_semana', 'nm_dia', 'flag_fim_semana', 'ds_mes_ano', 'ds_ano_trimestre']
n = load_dimension(df_dim_tempo, 'gold.dim_tmp', cols_tempo, cursor, conn)
print(f"   ✅ dim_tmp: {n} registros")

In [ ]:
# dim_loc
cols_loc = ['sg_uf', 'nm_uf', 'nm_regiao', 'cd_ibge', 'nm_capital']
n = load_dimension(df_dim_localizacao, 'gold.dim_loc', cols_loc, cursor, conn)
print(f"   ✅ dim_loc: {n} registros")

In [ ]:
# dim_pac
cols_pac = ['nk_demografica', 'ds_faixa_etaria', 'ds_sexo', 'ds_raca', 'ds_faixa_etaria_det']
n = load_dimension(df_dim_paciente, 'gold.dim_pac', cols_pac, cursor, conn)
print(f"   ✅ dim_pac: {n} registros")

In [ ]:
# dim_cls
cols_cls = ['cd_classificacao', 'ds_classificacao', 'ds_grupo', 'ds_gravidade', 'cd_cid', 'flag_confirmado']
n = load_dimension(df_dim_classificacao, 'gold.dim_cls', cols_cls, cursor, conn)
print(f"   ✅ dim_cls: {n} registros")

In [ ]:
# dim_evl
cols_evl = ['cd_evolucao', 'ds_evolucao', 'ds_tipo_evolucao', 'flag_obito', 'ds_gravidade_desfecho']
n = load_dimension(df_dim_evolucao, 'gold.dim_evl', cols_evl, cursor, conn)
print(f"   ✅ dim_evl: {n} registros")

In [ ]:
# dim_snt
cols_snt = ['nk_sintomas', 'ds_faixa_sintomas', 'ds_faixa_alarmes', 'ds_perfil_clinico', 'flag_tem_sintomas', 'flag_tem_alarmes']
n = load_dimension(df_dim_sintomas, 'gold.dim_snt', cols_snt, cursor, conn)
print(f"   ✅ dim_snt: {n} registros")

## 6. Criar Lookups para Foreign Keys

In [ ]:
print("⏳ Criando lookups para FKs...")

# Lookup dim_tmp
cursor.execute("SELECT sk_tmp, dt_completa FROM gold.dim_tmp")
df_lookup_tempo = pd.DataFrame(cursor.fetchall(), columns=['sk_tmp', 'dt_completa'])
lookup_tempo = dict(zip(df_lookup_tempo['dt_completa'], df_lookup_tempo['sk_tmp']))
print(f"   ✅ Lookup tempo: {len(lookup_tempo)} datas")

# Lookup dim_loc
cursor.execute("SELECT sk_loc, sg_uf FROM gold.dim_loc")
df_lookup_loc = pd.DataFrame(cursor.fetchall(), columns=['sk_loc', 'sg_uf'])
lookup_localizacao = dict(zip(df_lookup_loc['sg_uf'], df_lookup_loc['sk_loc']))
print(f"   ✅ Lookup localização: {len(lookup_localizacao)} UFs")

# Lookup dim_pac
cursor.execute("SELECT sk_pac, nk_demografica FROM gold.dim_pac")
df_lookup_pac = pd.DataFrame(cursor.fetchall(), columns=['sk_pac', 'nk_demografica'])
lookup_paciente = dict(zip(df_lookup_pac['nk_demografica'], df_lookup_pac['sk_pac']))
print(f"   ✅ Lookup paciente: {len(lookup_paciente)} perfis")

# Lookup dim_cls
cursor.execute("SELECT sk_cls, cd_classificacao FROM gold.dim_cls")
df_lookup_class = pd.DataFrame(cursor.fetchall(), columns=['sk_cls', 'cd_classificacao'])
lookup_classificacao = dict(zip(df_lookup_class['cd_classificacao'], df_lookup_class['sk_cls']))
print(f"   ✅ Lookup classificação: {len(lookup_classificacao)} tipos")

# Lookup dim_evl
cursor.execute("SELECT sk_evl, cd_evolucao FROM gold.dim_evl")
df_lookup_evol = pd.DataFrame(cursor.fetchall(), columns=['sk_evl', 'cd_evolucao'])
lookup_evolucao = dict(zip(df_lookup_evol['cd_evolucao'], df_lookup_evol['sk_evl']))
print(f"   ✅ Lookup evolução: {len(lookup_evolucao)} tipos")

# Lookup dim_snt
cursor.execute("SELECT sk_snt, nk_sintomas FROM gold.dim_snt")
df_lookup_sint = pd.DataFrame(cursor.fetchall(), columns=['sk_snt', 'nk_sintomas'])
lookup_sintomas = dict(zip(df_lookup_sint['nk_sintomas'], df_lookup_sint['sk_snt']))
print(f"   ✅ Lookup sintomas: {len(lookup_sintomas)} perfis")

## 7. Preparar DataFrame da Fato

In [ ]:
print("⏳ Preparando tabela fato...")

# Preparar chaves de lookup para o Silver
df_silver['nk_demografica'] = (df_paciente_unique['ds_faixa_etaria'] + '|' + 
                                df_paciente_unique['ds_sexo'] + '|' + 
                                df_paciente_unique['ds_raca'])

df_silver['nk_sintomas'] = (df_sintomas_unique['ds_faixa_sintomas'] + '|' + 
                             df_sintomas_unique['ds_faixa_alarmes'])

# Criar DataFrame da fato
df_fato = pd.DataFrame()
df_fato['nk_notificacao'] = df_silver['id_agravo']

# Foreign Keys
df_fato['fk_tmp'] = df_silver['dt_notific'].apply(
    lambda x: lookup_tempo.get(pd.to_datetime(x).date(), -1) if pd.notna(x) else -1
)

df_fato['fk_loc'] = df_silver['sg_uf_not'].apply(
    lambda x: lookup_localizacao.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_pac'] = df_silver['nk_demografica'].apply(
    lambda x: lookup_paciente.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_cls'] = df_silver['classi_fin'].apply(
    lambda x: lookup_classificacao.get(str(int(x)), -1) if pd.notna(x) else -1
)

df_fato['fk_evl'] = df_silver['evolucao'].apply(
    lambda x: lookup_evolucao.get(str(int(x)), -1) if pd.notna(x) else -1
)

df_fato['fk_snt'] = df_silver['nk_sintomas'].apply(
    lambda x: lookup_sintomas.get(x, -1) if pd.notna(x) else -1
)

# Métricas
df_fato['vl_confirmado'] = df_silver['classi_fin'].apply(
    lambda x: 1 if pd.notna(x) and int(x) in [10, 11, 12, 13] else 0
)

df_fato['vl_grave'] = df_silver['classi_fin'].apply(
    lambda x: 1 if pd.notna(x) and int(x) == 12 else 0
)

df_fato['vl_obito'] = df_silver['evolucao'].apply(
    lambda x: 1 if pd.notna(x) and int(x) in [2, 3, 4] else 0
)

df_fato['vl_hospitalizado'] = df_silver['hospitaliz'].apply(
    lambda x: 1 if pd.notna(x) and int(x) == 1 else 0
)

df_fato['vl_qtd_sintomas'] = df_silver['qtd_sintomas'].fillna(0).astype(int)
df_fato['vl_qtd_alarmes'] = df_silver['qtd_alarmes'].fillna(0).astype(int)
df_fato['vl_idade_anos'] = df_silver['idade_anos']

# Datas
df_fato['dt_notificacao'] = pd.to_datetime(df_silver['dt_notific']).dt.date
df_fato['dt_sintomas'] = pd.to_datetime(df_silver['dt_sin_pri']).dt.date

print(f"✅ Fato preparada: {len(df_fato):,} registros")
df_fato.head()

In [ ]:
# Verificar integridade das FKs antes da carga
print("🔍 Verificando integridade referencial...")
fk_checks = ['fk_tmp', 'fk_loc', 'fk_pac', 'fk_cls', 'fk_evl', 'fk_snt']
total_nulls = 0
for fk_col in fk_checks:
    null_count = (df_fato[fk_col] == -1).sum()
    total_nulls += null_count
    if null_count > 0:
        print(f"   ⚠️ {fk_col}: {null_count:,} registros com FK -1 (UNKNOWN)")
    else:
        print(f"   ✅ {fk_col}: OK")

print(f"\n📊 Total FKs com UNKNOWN: {total_nulls:,} ({total_nulls/len(df_fato)*100:.2f}%)")

## 8. Carga da Tabela Fato

In [ ]:
# Preparar dados finais para carga
df_fato_final = df_fato.copy()

# Garantir tipos corretos
df_fato_final['nk_notificacao'] = df_fato_final['nk_notificacao'].astype(int)
df_fato_final['fk_tmp'] = df_fato_final['fk_tmp'].astype(int)
df_fato_final['fk_loc'] = df_fato_final['fk_loc'].astype(int)
df_fato_final['fk_pac'] = df_fato_final['fk_pac'].astype(int)
df_fato_final['fk_cls'] = df_fato_final['fk_cls'].astype(int)
df_fato_final['fk_evl'] = df_fato_final['fk_evl'].astype(int)
df_fato_final['fk_snt'] = df_fato_final['fk_snt'].astype(int)

# Colunas para inserção
cols_fato = ['nk_notificacao', 'fk_tmp', 'fk_loc', 'fk_pac', 'fk_cls', 'fk_evl', 'fk_snt',
             'vl_confirmado', 'vl_grave', 'vl_obito', 'vl_hospitalizado', 
             'vl_qtd_sintomas', 'vl_qtd_alarmes', 'vl_idade_anos',
             'dt_notificacao', 'dt_sintomas']

# SQL de inserção
insert_sql = f"""
INSERT INTO gold.ft_deng 
({', '.join(cols_fato)})
VALUES %s
"""

print(f"⏳ Iniciando carga de {len(df_fato_final):,} registros em batches de {BATCH_SIZE:,}...")

# Carga em batches
total_rows = len(df_fato_final)
total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE

for batch_num in range(total_batches):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, total_rows)
    
    batch_df = df_fato_final.iloc[start_idx:end_idx]
    batch_data = [tuple(row[col] if pd.notna(row[col]) else None for col in cols_fato) 
                  for _, row in batch_df.iterrows()]
    
    execute_values(cursor, insert_sql, batch_data)
    conn.commit()
    
    progress = (batch_num + 1) / total_batches * 100
    print(f"   Batch {batch_num + 1}/{total_batches}: {end_idx:,} registros ({progress:.1f}%)")

print(f"\n✅ Carga concluída: {total_rows:,} registros inseridos")

## 9. Validações

In [ ]:
print("🔍 Validando carga...\n")

# 1. Volumetria
cursor.execute("SELECT COUNT(*) FROM gold.ft_deng")
count_gold = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM silver.dengue_notificacoes")
count_silver = cursor.fetchone()[0]

print(f"📊 VOLUMETRIA:")
print(f"   Silver: {count_silver:,}")
print(f"   Gold:   {count_gold:,}")
print(f"   Status: {'✅ OK' if count_gold == count_silver else '❌ DIFERENÇA'}")

# 2. Integridade referencial
print(f"\n🔗 INTEGRIDADE REFERENCIAL:")
for dim_table, fk_col, sk_col in [
    ('dim_tmp', 'fk_tmp', 'sk_tmp'),
    ('dim_loc', 'fk_loc', 'sk_loc'),
    ('dim_pac', 'fk_pac', 'sk_pac'),
    ('dim_cls', 'fk_cls', 'sk_cls'),
    ('dim_evl', 'fk_evl', 'sk_evl'),
    ('dim_snt', 'fk_snt', 'sk_snt')
]:
    query = f"""
    SELECT COUNT(*) FROM gold.ft_deng f
    LEFT JOIN gold.{dim_table} d ON f.{fk_col} = d.{sk_col}
    WHERE d.{sk_col} IS NULL
    """
    cursor.execute(query)
    orphans = cursor.fetchone()[0]
    status = '✅ OK' if orphans == 0 else f'❌ {orphans:,} órfãos'
    print(f"   {dim_table}: {status}")

In [ ]:
# 3. Métricas críticas
print(f"\n📈 MÉTRICAS CRÍTICAS:")
query = """
SELECT 
    SUM(vl_confirmado) as casos_confirmados,
    SUM(vl_grave) as casos_graves,
    SUM(vl_obito) as obitos,
    SUM(vl_hospitalizado) as hospitalizados
FROM gold.ft_deng
"""
cursor.execute(query)
metricas = pd.DataFrame([cursor.fetchone()], 
                        columns=['casos_confirmados', 'casos_graves', 'obitos', 'hospitalizados'])
print(metricas.T)

In [ ]:
# 4. Distribuição por região
print(f"\n🗺️ DISTRIBUIÇÃO POR REGIÃO:")
query = """
SELECT 
    d.nm_regiao,
    COUNT(*) as casos,
    SUM(f.vl_confirmado) as confirmados,
    SUM(f.vl_obito) as obitos
FROM gold.ft_deng f
JOIN gold.dim_loc d ON f.fk_loc = d.sk_loc
WHERE d.nm_regiao != 'UNKNOWN'
GROUP BY d.nm_regiao
ORDER BY casos DESC
"""
cursor.execute(query)
resultado = pd.DataFrame(cursor.fetchall(), 
                          columns=['nm_regiao', 'casos', 'confirmados', 'obitos'])
print(resultado)

In [ ]:
# 5. Taxa de letalidade
print(f"\n⚠️ TAXA DE LETALIDADE:")
query = """
SELECT 
    CASE WHEN SUM(vl_confirmado) > 0 
         THEN ROUND(SUM(vl_obito)::numeric / SUM(vl_confirmado) * 100, 4) 
         ELSE 0 END as taxa_letalidade_pct
FROM gold.ft_deng
"""
cursor.execute(query)
taxa = cursor.fetchone()[0]
print(f"   Taxa de letalidade: {taxa}%")

In [ ]:
# Fechar conexão
cursor.close()
conn.close()
print("\n✅ ETL Silver → Gold concluído com sucesso!")
print(f"   Total de registros carregados: {count_gold:,}")

# ETL Silver to Gold - Dengue Data Warehouse
## Pipeline Medallion: Star Schema com 1 Fato e 6 Dimensões

**Objetivo**: Transformar dados Silver em modelo dimensional Gold

**Modelo**: Star Schema
- 1 Tabela Fato: `ft_dengue`
- 6 Dimensões: `dim_tempo`, `dim_localizacao`, `dim_paciente`, `dim_classificacao`, `dim_evolucao`, `dim_sintomas`

**Tecnologia**: Pandas + PostgreSQL (conforme recomendado no documento)

**Volumetria**: 1.661.634 registros

## 1. Configuração Inicial

In [3]:
# Imports
import pandas as pd
import psycopg2
from psycopg2 import sql
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuração de conexão PostgreSQL
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'gis',
    'user': 'postgres',
    'password': 'postgres'
}

def get_connection():
    """Cria conexão com PostgreSQL"""
    return psycopg2.connect(**DB_CONFIG)

print("✅ Configuração carregada")

✅ Configuração carregada


## 2. Criação do Schema Gold (DDL)

In [4]:
# Executar DDL Gold Schema
ddl_path = '../Data_Layer/gold/ddl_gold_schema.sql'

with open(ddl_path, 'r') as f:
    ddl_script = f.read()

conn = get_connection()
cursor = conn.cursor()

try:
    # Executar cada comando separadamente
    for statement in ddl_script.split(';'):
        statement = statement.strip()
        if statement and not statement.startswith('--'):
            try:
                cursor.execute(statement)
            except Exception as e:
                # Ignorar erros de tabela já existente
                if 'already exists' not in str(e):
                    print(f"Aviso: {e}")
    conn.commit()
    print("✅ Schema Gold criado com sucesso")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro: {e}")
finally:
    cursor.close()
    conn.close()

Aviso: schema "gold" does not exist

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction block

Aviso: current transaction is aborted, commands ignored until end of transaction bl

## 3. Extração dos Dados Silver

In [5]:
# Extrair dados da camada Silver
query_silver = """
SELECT 
    id_notificacao,
    fl_confirmado,
    fl_grave,
    fl_obito,
    fl_hospitalizado,
    ano_notificacao,
    mes_notificacao,
    semana_epi,
    data_notificacao,
    data_sintomas,
    uf_sigla,
    faixa_etaria,
    sexo_desc,
    raca_desc,
    classificacao_desc,
    evolucao_desc,
    qtd_sintomas,
    qtd_alarmes,
    idade_anos
FROM public.dengue_silver
"""

conn = get_connection()
df_silver = pd.read_sql(query_silver, conn)
conn.close()

print(f"✅ Dados Silver extraídos: {len(df_silver):,} registros")
print(f"   Colunas: {list(df_silver.columns)}")

✅ Dados Silver extraídos: 1,661,634 registros
   Colunas: ['id_notificacao', 'fl_confirmado', 'fl_grave', 'fl_obito', 'fl_hospitalizado', 'ano_notificacao', 'mes_notificacao', 'semana_epi', 'data_notificacao', 'data_sintomas', 'uf_sigla', 'faixa_etaria', 'sexo_desc', 'raca_desc', 'classificacao_desc', 'evolucao_desc', 'qtd_sintomas', 'qtd_alarmes', 'idade_anos']


In [6]:
# Validar volumetria esperada
VOLUMETRIA_ESPERADA = 1661634

if len(df_silver) == VOLUMETRIA_ESPERADA:
    print(f"✅ Volumetria validada: {len(df_silver):,} registros")
else:
    print(f"⚠️ Volumetria diferente: esperado {VOLUMETRIA_ESPERADA:,}, obtido {len(df_silver):,}")

✅ Volumetria validada: 1,661,634 registros


## 4. Criação das Dimensões

### 4.1 dim_tempo (Temporal)

In [7]:
# Criar dimensão tempo a partir das datas de notificação
datas_unicas = df_silver['data_notificacao'].dropna().unique()

# Mapeamento dia da semana (ISO: Segunda=1)
DIAS_SEMANA = {
    0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta',
    4: 'Sexta', 5: 'Sábado', 6: 'Domingo'
}

dim_tempo_data = []
for data in datas_unicas:
    if pd.isna(data):
        continue
    d = pd.to_datetime(data)
    dia_semana_python = d.weekday()  # 0=Segunda
    
    dim_tempo_data.append({
        'data_completa': d.date(),
        'ano': d.year,
        'mes': d.month,
        'dia': d.day,
        'trimestre': (d.month - 1) // 3 + 1,
        'semana_epi': d.isocalendar()[1],  # Semana ISO
        'dia_semana': dia_semana_python + 1,  # ISO: Segunda=1
        'nome_dia': DIAS_SEMANA[dia_semana_python],
        'flag_fim_semana': dia_semana_python >= 5,  # Sábado=5, Domingo=6
        'mes_ano': f"{d.year}-{d.month:02d}",
        'ano_trimestre': f"{d.year}-Q{(d.month - 1) // 3 + 1}"
    })

df_dim_tempo = pd.DataFrame(dim_tempo_data)
df_dim_tempo = df_dim_tempo.drop_duplicates(subset=['data_completa'])
df_dim_tempo = df_dim_tempo.sort_values('data_completa').reset_index(drop=True)

# Gerar surrogate key (começando em 1)
df_dim_tempo['sk_tempo'] = range(1, len(df_dim_tempo) + 1)

print(f"✅ dim_tempo criada: {len(df_dim_tempo)} registros")
print(f"   Período: {df_dim_tempo['data_completa'].min()} a {df_dim_tempo['data_completa'].max()}")
df_dim_tempo.head()

✅ dim_tempo criada: 373 registros
   Período: 2024-12-29 a 2026-01-05


,data_completa,ano,mes,dia,trimestre,semana_epi,dia_semana,nome_dia,flag_fim_semana,mes_ano,ano_trimestre,sk_tempo
0,2024-12-29,2024,12,29,4,52,7,Domingo,True,2024-12,2024-Q4,1
1,2024-12-30,2024,12,30,4,1,1,Segunda,False,2024-12,2024-Q4,2
2,2024-12-31,2024,12,31,4,1,2,Terça,False,2024-12,2024-Q4,3
3,2025-01-01,2025,1,1,1,1,3,Quarta,False,2025-01,2025-Q1,4
4,2025-01-02,2025,1,2,1,1,4,Quinta,False,2025-01,2025-Q1,5


### 4.2 dim_localizacao (Geográfica)

In [8]:
# Dados de referência UFs brasileiras
UFS_BRASIL = {
    'AC': {'nome': 'Acre', 'regiao': 'Norte', 'ibge': 12, 'capital': 'Rio Branco'},
    'AL': {'nome': 'Alagoas', 'regiao': 'Nordeste', 'ibge': 27, 'capital': 'Maceió'},
    'AM': {'nome': 'Amazonas', 'regiao': 'Norte', 'ibge': 13, 'capital': 'Manaus'},
    'AP': {'nome': 'Amapá', 'regiao': 'Norte', 'ibge': 16, 'capital': 'Macapá'},
    'BA': {'nome': 'Bahia', 'regiao': 'Nordeste', 'ibge': 29, 'capital': 'Salvador'},
    'CE': {'nome': 'Ceará', 'regiao': 'Nordeste', 'ibge': 23, 'capital': 'Fortaleza'},
    'DF': {'nome': 'Distrito Federal', 'regiao': 'Centro-Oeste', 'ibge': 53, 'capital': 'Brasília'},
    'ES': {'nome': 'Espírito Santo', 'regiao': 'Sudeste', 'ibge': 32, 'capital': 'Vitória'},
    'GO': {'nome': 'Goiás', 'regiao': 'Centro-Oeste', 'ibge': 52, 'capital': 'Goiânia'},
    'MA': {'nome': 'Maranhão', 'regiao': 'Nordeste', 'ibge': 21, 'capital': 'São Luís'},
    'MG': {'nome': 'Minas Gerais', 'regiao': 'Sudeste', 'ibge': 31, 'capital': 'Belo Horizonte'},
    'MS': {'nome': 'Mato Grosso do Sul', 'regiao': 'Centro-Oeste', 'ibge': 50, 'capital': 'Campo Grande'},
    'MT': {'nome': 'Mato Grosso', 'regiao': 'Centro-Oeste', 'ibge': 51, 'capital': 'Cuiabá'},
    'PA': {'nome': 'Pará', 'regiao': 'Norte', 'ibge': 15, 'capital': 'Belém'},
    'PB': {'nome': 'Paraíba', 'regiao': 'Nordeste', 'ibge': 25, 'capital': 'João Pessoa'},
    'PE': {'nome': 'Pernambuco', 'regiao': 'Nordeste', 'ibge': 26, 'capital': 'Recife'},
    'PI': {'nome': 'Piauí', 'regiao': 'Nordeste', 'ibge': 22, 'capital': 'Teresina'},
    'PR': {'nome': 'Paraná', 'regiao': 'Sul', 'ibge': 41, 'capital': 'Curitiba'},
    'RJ': {'nome': 'Rio de Janeiro', 'regiao': 'Sudeste', 'ibge': 33, 'capital': 'Rio de Janeiro'},
    'RN': {'nome': 'Rio Grande do Norte', 'regiao': 'Nordeste', 'ibge': 24, 'capital': 'Natal'},
    'RO': {'nome': 'Rondônia', 'regiao': 'Norte', 'ibge': 11, 'capital': 'Porto Velho'},
    'RR': {'nome': 'Roraima', 'regiao': 'Norte', 'ibge': 14, 'capital': 'Boa Vista'},
    'RS': {'nome': 'Rio Grande do Sul', 'regiao': 'Sul', 'ibge': 43, 'capital': 'Porto Alegre'},
    'SC': {'nome': 'Santa Catarina', 'regiao': 'Sul', 'ibge': 42, 'capital': 'Florianópolis'},
    'SE': {'nome': 'Sergipe', 'regiao': 'Nordeste', 'ibge': 28, 'capital': 'Aracaju'},
    'SP': {'nome': 'São Paulo', 'regiao': 'Sudeste', 'ibge': 35, 'capital': 'São Paulo'},
    'TO': {'nome': 'Tocantins', 'regiao': 'Norte', 'ibge': 17, 'capital': 'Palmas'}
}

# Criar dimensão localização
ufs_silver = df_silver['uf_sigla'].dropna().unique()

dim_localizacao_data = []
for uf in ufs_silver:
    if uf in UFS_BRASIL:
        info = UFS_BRASIL[uf]
        dim_localizacao_data.append({
            'uf_sigla': uf,
            'uf_nome': info['nome'],
            'regiao': info['regiao'],
            'codigo_ibge': info['ibge'],
            'capital': info['capital']
        })

df_dim_localizacao = pd.DataFrame(dim_localizacao_data)
df_dim_localizacao = df_dim_localizacao.sort_values('uf_sigla').reset_index(drop=True)

# Gerar surrogate key (começando em 1)
df_dim_localizacao['sk_localizacao'] = range(1, len(df_dim_localizacao) + 1)

print(f"✅ dim_localizacao criada: {len(df_dim_localizacao)} UFs")
print(f"   Regiões: {df_dim_localizacao['regiao'].unique().tolist()}")
df_dim_localizacao

✅ dim_localizacao criada: 27 UFs
   Regiões: ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']


,uf_sigla,uf_nome,regiao,codigo_ibge,capital,sk_localizacao
0,AC,Acre,Norte,12,Rio Branco,1
1,AL,Alagoas,Nordeste,27,Maceió,2
2,AM,Amazonas,Norte,13,Manaus,3
3,AP,Amapá,Norte,16,Macapá,4
4,BA,Bahia,Nordeste,29,Salvador,5
5,CE,Ceará,Nordeste,23,Fortaleza,6
6,DF,Distrito Federal,Centro-Oeste,53,Brasília,7
7,ES,Espírito Santo,Sudeste,32,Vitória,8
8,GO,Goiás,Centro-Oeste,52,Goiânia,9
9,MA,Maranhão,Nordeste,21,São Luís,10


### 4.3 dim_paciente (Demográfica)

In [9]:
# Criar combinações demográficas únicas
df_paciente_unique = df_silver[['faixa_etaria', 'sexo_desc', 'raca_desc']].drop_duplicates()

# Criar business key (combinação demográfica)
df_paciente_unique['combinacao_demografica'] = (
    df_paciente_unique['faixa_etaria'].fillna('UNKNOWN') + '|' +
    df_paciente_unique['sexo_desc'].fillna('UNKNOWN') + '|' +
    df_paciente_unique['raca_desc'].fillna('UNKNOWN')
)

# Faixa etária detalhada (conforme documento: Subcategorias mais granulares)
def get_faixa_etaria_detalhada(faixa):
    if pd.isna(faixa):
        return 'Não informado'
    faixa_str = str(faixa)
    # Mapeamento para subcategorias
    if '< 1' in faixa_str or 'menor' in faixa_str.lower():
        return 'Lactente (< 1 ano)'
    elif '1-4' in faixa_str:
        return 'Pré-escolar (1-4 anos)'
    elif '5-9' in faixa_str:
        return 'Escolar (5-9 anos)'
    elif '10-14' in faixa_str:
        return 'Adolescente inicial (10-14 anos)'
    elif '15-19' in faixa_str:
        return 'Adolescente final (15-19 anos)'
    elif '20-39' in faixa_str:
        return 'Adulto jovem (20-39 anos)'
    elif '40-59' in faixa_str:
        return 'Adulto (40-59 anos)'
    elif '60+' in faixa_str or '>=' in faixa_str or 'idoso' in faixa_str.lower():
        return 'Idoso (60+ anos)'
    else:
        return faixa_str

df_paciente_unique['faixa_etaria_detalhada'] = df_paciente_unique['faixa_etaria'].apply(get_faixa_etaria_detalhada)

# Ordenar e criar surrogate key
df_dim_paciente = df_paciente_unique.sort_values('combinacao_demografica').reset_index(drop=True)
df_dim_paciente['sk_paciente'] = range(1, len(df_dim_paciente) + 1)

# Preencher valores nulos
df_dim_paciente['faixa_etaria'] = df_dim_paciente['faixa_etaria'].fillna('UNKNOWN')
df_dim_paciente['sexo_desc'] = df_dim_paciente['sexo_desc'].fillna('UNKNOWN')
df_dim_paciente['raca_desc'] = df_dim_paciente['raca_desc'].fillna('UNKNOWN')

print(f"✅ dim_paciente criada: {len(df_dim_paciente)} combinações demográficas")
print(f"   Faixas etárias: {df_dim_paciente['faixa_etaria'].nunique()}")
print(f"   Sexos: {df_dim_paciente['sexo_desc'].unique().tolist()}")
df_dim_paciente.head(10)

✅ dim_paciente criada: 123 combinações demográficas
   Faixas etárias: 8
   Sexos: ['Feminino', 'Ignorado', 'Masculino', 'Nao informado']


,faixa_etaria,sexo_desc,raca_desc,combinacao_demografica,faixa_etaria_detalhada,sk_paciente
0,1-4 anos,Feminino,Amarela,1-4 anos|Feminino|Amarela,Pré-escolar (1-4 anos),1
1,1-4 anos,Feminino,Branca,1-4 anos|Feminino|Branca,Pré-escolar (1-4 anos),2
2,1-4 anos,Feminino,Ignorado,1-4 anos|Feminino|Ignorado,Pré-escolar (1-4 anos),3
3,1-4 anos,Feminino,Indigena,1-4 anos|Feminino|Indigena,Pré-escolar (1-4 anos),4
4,1-4 anos,Feminino,Parda,1-4 anos|Feminino|Parda,Pré-escolar (1-4 anos),5
5,1-4 anos,Feminino,Preta,1-4 anos|Feminino|Preta,Pré-escolar (1-4 anos),6
6,1-4 anos,Ignorado,Branca,1-4 anos|Ignorado|Branca,Pré-escolar (1-4 anos),7
7,1-4 anos,Ignorado,Ignorado,1-4 anos|Ignorado|Ignorado,Pré-escolar (1-4 anos),8
8,1-4 anos,Ignorado,Parda,1-4 anos|Ignorado|Parda,Pré-escolar (1-4 anos),9
9,1-4 anos,Masculino,Amarela,1-4 anos|Masculino|Amarela,Pré-escolar (1-4 anos),10


### 4.4 dim_classificacao (Clínica/Epidemiológica)

In [10]:
# Classificações únicas do Silver
classificacoes = df_silver['classificacao_desc'].dropna().unique()

# Mapeamentos conforme documento
def get_classificacao_grupo(desc):
    if pd.isna(desc):
        return 'UNKNOWN'
    desc_lower = str(desc).lower()
    if 'descart' in desc_lower:
        return 'Descartado'
    elif 'investig' in desc_lower or 'inconclu' in desc_lower:
        return 'Em Investigação'
    else:
        return 'Confirmado'

def get_gravidade(desc):
    if pd.isna(desc):
        return 'UNKNOWN'
    desc_lower = str(desc).lower()
    if 'grave' in desc_lower:
        return 'Grave'
    elif 'alarme' in desc_lower:
        return 'Moderado'
    else:
        return 'Leve'

def get_codigo_cid(desc):
    """Código CID-10 relacionado"""
    if pd.isna(desc):
        return None
    desc_lower = str(desc).lower()
    if 'grave' in desc_lower:
        return 'A91.0'  # Febre hemorrágica da dengue
    elif 'alarme' in desc_lower:
        return 'A90'    # Dengue clássico
    elif 'dengue' in desc_lower:
        return 'A90'    # Dengue clássico
    else:
        return None

def is_confirmado(desc):
    if pd.isna(desc):
        return False
    desc_lower = str(desc).lower()
    return 'dengue' in desc_lower and 'descart' not in desc_lower

dim_classificacao_data = []
for classif in classificacoes:
    dim_classificacao_data.append({
        'classificacao_codigo': str(classif),
        'classificacao_desc': str(classif),
        'classificacao_grupo': get_classificacao_grupo(classif),
        'gravidade': get_gravidade(classif),
        'codigo_cid': get_codigo_cid(classif),
        'flag_confirmado': is_confirmado(classif)
    })

df_dim_classificacao = pd.DataFrame(dim_classificacao_data)
df_dim_classificacao = df_dim_classificacao.sort_values('classificacao_desc').reset_index(drop=True)
df_dim_classificacao['sk_classificacao'] = range(1, len(df_dim_classificacao) + 1)

print(f"✅ dim_classificacao criada: {len(df_dim_classificacao)} classificações")
df_dim_classificacao

✅ dim_classificacao criada: 5 classificações


,classificacao_codigo,classificacao_desc,classificacao_grupo,gravidade,codigo_cid,flag_confirmado,sk_classificacao
0,Dengue,Dengue,Confirmado,Leve,A90,True,1
1,Dengue Grave,Dengue Grave,Confirmado,Grave,A91.0,True,2
2,Dengue com Sinais de Alarme,Dengue com Sinais de Alarme,Confirmado,Moderado,A90,True,3
3,Em investigacao,Em investigacao,Em Investigação,Leve,None,False,4
4,Inconclusivo,Inconclusivo,Em Investigação,Leve,None,False,5


### 4.5 dim_evolucao (Desfecho Clínico)

In [11]:
# Evoluções únicas do Silver
evolucoes = df_silver['evolucao_desc'].dropna().unique()

def get_tipo_evolucao(desc):
    if pd.isna(desc):
        return 'UNKNOWN'
    desc_lower = str(desc).lower()
    if 'óbito' in desc_lower or 'obito' in desc_lower:
        return 'Óbito'
    elif 'cura' in desc_lower:
        return 'Cura'
    else:
        return 'Em investigação'

def is_obito(desc):
    if pd.isna(desc):
        return False
    desc_lower = str(desc).lower()
    return 'óbito' in desc_lower or 'obito' in desc_lower

def get_gravidade_desfecho(desc):
    if pd.isna(desc):
        return 'Indeterminado'
    desc_lower = str(desc).lower()
    if 'óbito' in desc_lower or 'obito' in desc_lower:
        return 'Desfavorável'
    elif 'cura' in desc_lower:
        return 'Favorável'
    else:
        return 'Indeterminado'

dim_evolucao_data = []
for evol in evolucoes:
    dim_evolucao_data.append({
        'evolucao_codigo': str(evol),
        'evolucao_desc': str(evol),
        'tipo_evolucao': get_tipo_evolucao(evol),
        'flag_obito': is_obito(evol),
        'gravidade_desfecho': get_gravidade_desfecho(evol)
    })

df_dim_evolucao = pd.DataFrame(dim_evolucao_data)
df_dim_evolucao = df_dim_evolucao.sort_values('evolucao_desc').reset_index(drop=True)
df_dim_evolucao['sk_evolucao'] = range(1, len(df_dim_evolucao) + 1)

print(f"✅ dim_evolucao criada: {len(df_dim_evolucao)} evoluções")
df_dim_evolucao

✅ dim_evolucao criada: 6 evoluções


,evolucao_codigo,evolucao_desc,tipo_evolucao,flag_obito,gravidade_desfecho,sk_evolucao
0,Cura,Cura,Cura,False,Favorável,1
1,Em investigacao,Em investigacao,Em investigação,False,Indeterminado,2
2,Ignorado,Ignorado,Em investigação,False,Indeterminado,3
3,Obito em investigacao,Obito em investigacao,Óbito,True,Desfavorável,4
4,Obito pelo agravo,Obito pelo agravo,Óbito,True,Desfavorável,5
5,Obito por outras causas,Obito por outras causas,Óbito,True,Desfavorável,6


### 4.6 dim_sintomas (Agregada)

In [13]:
# Criar faixas de sintomas conforme documento
def get_faixa_sintomas(qtd):
    """Categorias: 0, 1-2, 3-5, 6+ sintomas"""
    if pd.isna(qtd):
        return 'UNKNOWN'
    qtd = int(qtd)
    if qtd == 0:
        return '0'
    elif qtd <= 2:
        return '1-2'
    elif qtd <= 5:
        return '3-5'
    else:
        return '6+'

def get_faixa_alarmes(qtd):
    """Categorias: 0, 1, 2+, 3+ alarmes"""
    if pd.isna(qtd):
        return 'UNKNOWN'
    qtd = int(qtd)
    if qtd == 0:
        return '0'
    elif qtd == 1:
        return '1'
    elif qtd == 2:
        return '2+'
    else:
        return '3+'

def get_perfil_clinico(qtd_sint, qtd_alarm):
    """Perfis: Assintomatico, Leve, Moderado, Grave"""
    if pd.isna(qtd_sint) or pd.isna(qtd_alarm):
        return 'UNKNOWN'
    qtd_sint = int(qtd_sint)
    qtd_alarm = int(qtd_alarm)
    
    if qtd_sint == 0:
        return 'Assintomático'
    elif qtd_alarm >= 3:
        return 'Grave'
    elif qtd_alarm >= 1:
        return 'Moderado'
    else:
        return 'Leve'

# Criar combinações únicas de sintomas
df_sintomas_unique = df_silver[['qtd_sintomas', 'qtd_alarmes']].drop_duplicates()

df_sintomas_unique['qtd_sintomas_faixa'] = df_sintomas_unique['qtd_sintomas'].apply(get_faixa_sintomas)
df_sintomas_unique['qtd_alarmes_faixa'] = df_sintomas_unique['qtd_alarmes'].apply(get_faixa_alarmes)
df_sintomas_unique['perfil_clinico'] = df_sintomas_unique.apply(
    lambda row: get_perfil_clinico(row['qtd_sintomas'], row['qtd_alarmes']), axis=1
)
df_sintomas_unique['flag_tem_sintomas'] = df_sintomas_unique['qtd_sintomas'].fillna(0).astype(int) > 0
df_sintomas_unique['flag_tem_alarmes'] = df_sintomas_unique['qtd_alarmes'].fillna(0).astype(int) > 0

# Business key
df_sintomas_unique['combinacao_sintomas'] = (
    df_sintomas_unique['qtd_sintomas_faixa'] + '|' +
    df_sintomas_unique['qtd_alarmes_faixa']
)

# Agrupar por combinação única de faixas
df_dim_sintomas = df_sintomas_unique[[
    'combinacao_sintomas', 'qtd_sintomas_faixa', 'qtd_alarmes_faixa',
    'perfil_clinico', 'flag_tem_sintomas', 'flag_tem_alarmes'
]].drop_duplicates()

df_dim_sintomas = df_dim_sintomas.sort_values('combinacao_sintomas').reset_index(drop=True)
df_dim_sintomas['sk_sintomas'] = range(1, len(df_dim_sintomas) + 1)

print(f"✅ dim_sintomas criada: {len(df_dim_sintomas)} combinações")
df_dim_sintomas

✅ dim_sintomas criada: 16 combinações


,combinacao_sintomas,qtd_sintomas_faixa,qtd_alarmes_faixa,perfil_clinico,flag_tem_sintomas,flag_tem_alarmes,sk_sintomas
0,0|0,0,0,Assintomático,False,False,1
1,0|1,0,1,Assintomático,False,True,2
2,0|2+,0,2+,Assintomático,False,True,3
3,0|3+,0,3+,Assintomático,False,True,4
4,1-2|0,1-2,0,Leve,True,False,5
5,1-2|1,1-2,1,Moderado,True,True,6
6,1-2|2+,1-2,2+,Moderado,True,True,7
7,1-2|3+,1-2,3+,Grave,True,True,8
8,3-5|0,3-5,0,Leve,True,False,9
9,3-5|1,3-5,1,Moderado,True,True,10


## 5. Carga das Dimensões no PostgreSQL

In [14]:
from io import StringIO

def truncate_table(cursor, table_name):
    """Limpa tabela mantendo registro UNKNOWN"""
    cursor.execute(f"DELETE FROM {table_name} WHERE sk_{table_name.split('.')[-1].replace('dim_', '')} > 0 OR sk_{table_name.split('.')[-1].replace('dim_', '')} IS NULL")

def load_dimension(conn, df, table_name, columns):
    """Carrega dimensão via COPY (mais eficiente)"""
    cursor = conn.cursor()
    
    # Preparar dados
    df_load = df[columns].copy()
    
    # Criar buffer
    buffer = StringIO()
    df_load.to_csv(buffer, index=False, header=False, sep='\t', na_rep='\\N')
    buffer.seek(0)
    
    # Carregar via COPY
    cursor.copy_from(buffer, table_name, sep='\t', null='\\N', columns=columns)
    
    return len(df_load)

print("Iniciando carga das dimensões...")

Iniciando carga das dimensões...


In [15]:
# Carga dim_tempo
conn = get_connection()
cursor = conn.cursor()

try:
    # Limpar dados existentes (exceto UNKNOWN)
    cursor.execute("DELETE FROM gold.dim_tempo WHERE sk_tempo > 0")
    
    # Reset sequence
    cursor.execute("SELECT setval('gold.dim_tempo_sk_tempo_seq', 1, false)")
    
    # Inserir dados
    cols_tempo = ['data_completa', 'ano', 'mes', 'dia', 'trimestre', 'semana_epi',
                  'dia_semana', 'nome_dia', 'flag_fim_semana', 'mes_ano', 'ano_trimestre']
    
    for _, row in df_dim_tempo.iterrows():
        cursor.execute("""
            INSERT INTO gold.dim_tempo (data_completa, ano, mes, dia, trimestre, semana_epi,
                                        dia_semana, nome_dia, flag_fim_semana, mes_ano, ano_trimestre)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (row['data_completa'], row['ano'], row['mes'], row['dia'], row['trimestre'],
              row['semana_epi'], row['dia_semana'], row['nome_dia'], row['flag_fim_semana'],
              row['mes_ano'], row['ano_trimestre']))
    
    conn.commit()
    print(f"✅ dim_tempo carregada: {len(df_dim_tempo)} registros")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro dim_tempo: {e}")
finally:
    cursor.close()
    conn.close()

✅ dim_tempo carregada: 373 registros


In [16]:
# Carga dim_localizacao
conn = get_connection()
cursor = conn.cursor()

try:
    cursor.execute("DELETE FROM gold.dim_localizacao WHERE sk_localizacao > 0")
    cursor.execute("SELECT setval('gold.dim_localizacao_sk_localizacao_seq', 1, false)")
    
    for _, row in df_dim_localizacao.iterrows():
        cursor.execute("""
            INSERT INTO gold.dim_localizacao (uf_sigla, uf_nome, regiao, codigo_ibge, capital)
            VALUES (%s, %s, %s, %s, %s)
        """, (row['uf_sigla'], row['uf_nome'], row['regiao'], row['codigo_ibge'], row['capital']))
    
    conn.commit()
    print(f"✅ dim_localizacao carregada: {len(df_dim_localizacao)} registros")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro dim_localizacao: {e}")
finally:
    cursor.close()
    conn.close()

✅ dim_localizacao carregada: 27 registros


In [17]:
# Carga dim_paciente
conn = get_connection()
cursor = conn.cursor()

try:
    cursor.execute("DELETE FROM gold.dim_paciente WHERE sk_paciente > 0")
    cursor.execute("SELECT setval('gold.dim_paciente_sk_paciente_seq', 1, false)")
    
    for _, row in df_dim_paciente.iterrows():
        cursor.execute("""
            INSERT INTO gold.dim_paciente (combinacao_demografica, faixa_etaria, sexo_desc, raca_desc, faixa_etaria_detalhada)
            VALUES (%s, %s, %s, %s, %s)
        """, (row['combinacao_demografica'], row['faixa_etaria'], row['sexo_desc'], 
              row['raca_desc'], row['faixa_etaria_detalhada']))
    
    conn.commit()
    print(f"✅ dim_paciente carregada: {len(df_dim_paciente)} registros")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro dim_paciente: {e}")
finally:
    cursor.close()
    conn.close()

✅ dim_paciente carregada: 123 registros


In [18]:
# Carga dim_classificacao
conn = get_connection()
cursor = conn.cursor()

try:
    cursor.execute("DELETE FROM gold.dim_classificacao WHERE sk_classificacao > 0")
    cursor.execute("SELECT setval('gold.dim_classificacao_sk_classificacao_seq', 1, false)")
    
    for _, row in df_dim_classificacao.iterrows():
        cursor.execute("""
            INSERT INTO gold.dim_classificacao (classificacao_codigo, classificacao_desc, 
                                                 classificacao_grupo, gravidade, codigo_cid, flag_confirmado)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['classificacao_codigo'], row['classificacao_desc'], row['classificacao_grupo'],
              row['gravidade'], row['codigo_cid'], row['flag_confirmado']))
    
    conn.commit()
    print(f"✅ dim_classificacao carregada: {len(df_dim_classificacao)} registros")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro dim_classificacao: {e}")
finally:
    cursor.close()
    conn.close()

✅ dim_classificacao carregada: 5 registros


In [19]:
# Carga dim_evolucao
conn = get_connection()
cursor = conn.cursor()

try:
    cursor.execute("DELETE FROM gold.dim_evolucao WHERE sk_evolucao > 0")
    cursor.execute("SELECT setval('gold.dim_evolucao_sk_evolucao_seq', 1, false)")
    
    for _, row in df_dim_evolucao.iterrows():
        cursor.execute("""
            INSERT INTO gold.dim_evolucao (evolucao_codigo, evolucao_desc, tipo_evolucao, 
                                           flag_obito, gravidade_desfecho)
            VALUES (%s, %s, %s, %s, %s)
        """, (row['evolucao_codigo'], row['evolucao_desc'], row['tipo_evolucao'],
              row['flag_obito'], row['gravidade_desfecho']))
    
    conn.commit()
    print(f"✅ dim_evolucao carregada: {len(df_dim_evolucao)} registros")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro dim_evolucao: {e}")
finally:
    cursor.close()
    conn.close()

✅ dim_evolucao carregada: 6 registros


In [20]:
# Carga dim_sintomas
conn = get_connection()
cursor = conn.cursor()

try:
    cursor.execute("DELETE FROM gold.dim_sintomas WHERE sk_sintomas > 0")
    cursor.execute("SELECT setval('gold.dim_sintomas_sk_sintomas_seq', 1, false)")
    
    for _, row in df_dim_sintomas.iterrows():
        cursor.execute("""
            INSERT INTO gold.dim_sintomas (combinacao_sintomas, qtd_sintomas_faixa, qtd_alarmes_faixa,
                                           perfil_clinico, flag_tem_sintomas, flag_tem_alarmes)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['combinacao_sintomas'], row['qtd_sintomas_faixa'], row['qtd_alarmes_faixa'],
              row['perfil_clinico'], row['flag_tem_sintomas'], row['flag_tem_alarmes']))
    
    conn.commit()
    print(f"✅ dim_sintomas carregada: {len(df_dim_sintomas)} registros")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro dim_sintomas: {e}")
finally:
    cursor.close()
    conn.close()

✅ dim_sintomas carregada: 16 registros


## 6. Criar Lookups para Tabela Fato

In [21]:
# Recarregar dimensões do banco para obter SKs corretos
conn = get_connection()

# Lookup tempo: data_completa -> sk_tempo
df_lookup_tempo = pd.read_sql("SELECT sk_tempo, data_completa FROM gold.dim_tempo", conn)
lookup_tempo = dict(zip(df_lookup_tempo['data_completa'], df_lookup_tempo['sk_tempo']))

# Lookup localizacao: uf_sigla -> sk_localizacao
df_lookup_loc = pd.read_sql("SELECT sk_localizacao, uf_sigla FROM gold.dim_localizacao", conn)
lookup_localizacao = dict(zip(df_lookup_loc['uf_sigla'], df_lookup_loc['sk_localizacao']))

# Lookup paciente: combinacao_demografica -> sk_paciente
df_lookup_pac = pd.read_sql("SELECT sk_paciente, combinacao_demografica FROM gold.dim_paciente", conn)
lookup_paciente = dict(zip(df_lookup_pac['combinacao_demografica'], df_lookup_pac['sk_paciente']))

# Lookup classificacao: classificacao_codigo -> sk_classificacao
df_lookup_class = pd.read_sql("SELECT sk_classificacao, classificacao_codigo FROM gold.dim_classificacao", conn)
lookup_classificacao = dict(zip(df_lookup_class['classificacao_codigo'], df_lookup_class['sk_classificacao']))

# Lookup evolucao: evolucao_codigo -> sk_evolucao
df_lookup_evol = pd.read_sql("SELECT sk_evolucao, evolucao_codigo FROM gold.dim_evolucao", conn)
lookup_evolucao = dict(zip(df_lookup_evol['evolucao_codigo'], df_lookup_evol['sk_evolucao']))

# Lookup sintomas: combinacao_sintomas -> sk_sintomas
df_lookup_sint = pd.read_sql("SELECT sk_sintomas, combinacao_sintomas FROM gold.dim_sintomas", conn)
lookup_sintomas = dict(zip(df_lookup_sint['combinacao_sintomas'], df_lookup_sint['sk_sintomas']))

conn.close()

print(f"✅ Lookups criados:")
print(f"   - tempo: {len(lookup_tempo)} registros")
print(f"   - localizacao: {len(lookup_localizacao)} registros")
print(f"   - paciente: {len(lookup_paciente)} registros")
print(f"   - classificacao: {len(lookup_classificacao)} registros")
print(f"   - evolucao: {len(lookup_evolucao)} registros")
print(f"   - sintomas: {len(lookup_sintomas)} registros")

✅ Lookups criados:
   - tempo: 374 registros
   - localizacao: 28 registros
   - paciente: 124 registros
   - classificacao: 6 registros
   - evolucao: 7 registros
   - sintomas: 17 registros


## 7. Transformação e Carga da Tabela Fato

In [22]:
# Preparar DataFrame da fato
print("Preparando tabela fato...")

df_fato = df_silver.copy()

# FK Tempo (data_notificacao -> sk_tempo)
df_fato['fk_tempo'] = df_fato['data_notificacao'].map(lookup_tempo)
# Usar -1 (UNKNOWN) para valores não encontrados
df_fato['fk_tempo'] = df_fato['fk_tempo'].fillna(-1).astype(int)

# FK Localização (uf_sigla -> sk_localizacao)
df_fato['fk_localizacao'] = df_fato['uf_sigla'].map(lookup_localizacao)
df_fato['fk_localizacao'] = df_fato['fk_localizacao'].fillna(-1).astype(int)

# FK Paciente (combinação demográfica -> sk_paciente)
df_fato['combinacao_demografica'] = (
    df_fato['faixa_etaria'].fillna('UNKNOWN') + '|' +
    df_fato['sexo_desc'].fillna('UNKNOWN') + '|' +
    df_fato['raca_desc'].fillna('UNKNOWN')
)
df_fato['fk_paciente'] = df_fato['combinacao_demografica'].map(lookup_paciente)
df_fato['fk_paciente'] = df_fato['fk_paciente'].fillna(-1).astype(int)

# FK Classificação (classificacao_desc -> sk_classificacao)
df_fato['fk_classificacao'] = df_fato['classificacao_desc'].astype(str).map(lookup_classificacao)
df_fato['fk_classificacao'] = df_fato['fk_classificacao'].fillna(-1).astype(int)

# FK Evolução (evolucao_desc -> sk_evolucao)
df_fato['fk_evolucao'] = df_fato['evolucao_desc'].astype(str).map(lookup_evolucao)
df_fato['fk_evolucao'] = df_fato['fk_evolucao'].fillna(-1).astype(int)

# FK Sintomas (combinação sintomas -> sk_sintomas)
df_fato['qtd_sintomas_faixa'] = df_fato['qtd_sintomas'].apply(get_faixa_sintomas)
df_fato['qtd_alarmes_faixa'] = df_fato['qtd_alarmes'].apply(get_faixa_alarmes)
df_fato['combinacao_sintomas'] = df_fato['qtd_sintomas_faixa'] + '|' + df_fato['qtd_alarmes_faixa']
df_fato['fk_sintomas'] = df_fato['combinacao_sintomas'].map(lookup_sintomas)
df_fato['fk_sintomas'] = df_fato['fk_sintomas'].fillna(-1).astype(int)

# Métricas (conforme documento)
df_fato['vl_caso_confirmado'] = df_fato['fl_confirmado'].fillna(0).astype(int)
df_fato['vl_caso_grave'] = df_fato['fl_grave'].fillna(0).astype(int)
df_fato['vl_obito'] = df_fato['fl_obito'].fillna(0).astype(int)
df_fato['vl_hospitalizado'] = df_fato['fl_hospitalizado'].fillna(0).astype(int)
df_fato['vl_qtd_sintomas'] = df_fato['qtd_sintomas'].fillna(0).astype(int)
df_fato['vl_qtd_alarmes'] = df_fato['qtd_alarmes'].fillna(0).astype(int)
df_fato['vl_idade_anos'] = df_fato['idade_anos']  # Pode ser NULL

# Timestamps
df_fato['ts_notificacao'] = df_fato['data_notificacao']
df_fato['ts_sintomas'] = df_fato['data_sintomas']  # Pode ser NULL

print(f"✅ Tabela fato preparada: {len(df_fato):,} registros")

# Verificar FKs órfãs
print(f"\n📊 Verificação de FKs:")
print(f"   - fk_tempo=-1 (UNKNOWN): {(df_fato['fk_tempo'] == -1).sum():,}")
print(f"   - fk_localizacao=-1 (UNKNOWN): {(df_fato['fk_localizacao'] == -1).sum():,}")
print(f"   - fk_paciente=-1 (UNKNOWN): {(df_fato['fk_paciente'] == -1).sum():,}")
print(f"   - fk_classificacao=-1 (UNKNOWN): {(df_fato['fk_classificacao'] == -1).sum():,}")
print(f"   - fk_evolucao=-1 (UNKNOWN): {(df_fato['fk_evolucao'] == -1).sum():,}")
print(f"   - fk_sintomas=-1 (UNKNOWN): {(df_fato['fk_sintomas'] == -1).sum():,}")

Preparando tabela fato...
✅ Tabela fato preparada: 1,661,634 registros

📊 Verificação de FKs:
   - fk_tempo=-1 (UNKNOWN): 0
   - fk_localizacao=-1 (UNKNOWN): 0
   - fk_paciente=-1 (UNKNOWN): 0
   - fk_classificacao=-1 (UNKNOWN): 0
   - fk_evolucao=-1 (UNKNOWN): 0
   - fk_sintomas=-1 (UNKNOWN): 0


In [23]:
# Selecionar colunas finais da fato
cols_fato = [
    'id_notificacao',  # id_notificacao_original
    'fk_tempo',
    'fk_localizacao',
    'fk_paciente',
    'fk_classificacao',
    'fk_evolucao',
    'fk_sintomas',
    'vl_caso_confirmado',
    'vl_caso_grave',
    'vl_obito',
    'vl_hospitalizado',
    'vl_qtd_sintomas',
    'vl_qtd_alarmes',
    'vl_idade_anos',
    'ts_notificacao',
    'ts_sintomas'
]

df_fato_final = df_fato[cols_fato].copy()
df_fato_final = df_fato_final.rename(columns={'id_notificacao': 'id_notificacao_original'})

print(f"✅ DataFrame fato final: {len(df_fato_final):,} registros, {len(cols_fato)} colunas")
df_fato_final.head()

✅ DataFrame fato final: 1,661,634 registros, 16 colunas


,id_notificacao_original,fk_tempo,fk_localizacao,fk_paciente,fk_classificacao,fk_evolucao,fk_sintomas,vl_caso_confirmado,vl_caso_grave,vl_obito,vl_hospitalizado,vl_qtd_sintomas,vl_qtd_alarmes,vl_idade_anos,ts_notificacao,ts_sintomas
0,533023,70,18,49,1,1,9,1,0,0,0,5,0,20.0,2025-03-08,2025-03-04
1,533024,70,18,86,1,1,13,1,0,0,0,6,0,67.0,2025-03-08,2025-03-04
2,533025,72,18,34,1,1,13,1,0,0,0,6,0,20.0,2025-03-10,2025-03-08
3,533026,75,18,28,1,1,9,1,0,0,0,5,0,12.0,2025-03-13,2025-03-11
4,533027,75,18,86,1,1,13,1,0,0,0,6,0,60.0,2025-03-13,2025-03-07


In [24]:
# Carga da tabela fato em lotes (para performance)
BATCH_SIZE = 50000

conn = get_connection()
cursor = conn.cursor()

try:
    # Limpar tabela fato
    print("Limpando tabela fato...")
    cursor.execute("TRUNCATE TABLE gold.ft_dengue RESTART IDENTITY")
    conn.commit()
    
    # Inserir em lotes
    total_rows = len(df_fato_final)
    total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
    
    print(f"Inserindo {total_rows:,} registros em {total_batches} lotes...")
    
    insert_sql = """
        INSERT INTO gold.ft_dengue (
            id_notificacao_original, fk_tempo, fk_localizacao, fk_paciente,
            fk_classificacao, fk_evolucao, fk_sintomas,
            vl_caso_confirmado, vl_caso_grave, vl_obito, vl_hospitalizado,
            vl_qtd_sintomas, vl_qtd_alarmes, vl_idade_anos,
            ts_notificacao, ts_sintomas
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    for batch_num in range(total_batches):
        start_idx = batch_num * BATCH_SIZE
        end_idx = min(start_idx + BATCH_SIZE, total_rows)
        
        batch_df = df_fato_final.iloc[start_idx:end_idx]
        
        # Preparar dados do lote
        batch_data = []
        for _, row in batch_df.iterrows():
            batch_data.append((
                int(row['id_notificacao_original']),
                int(row['fk_tempo']),
                int(row['fk_localizacao']),
                int(row['fk_paciente']),
                int(row['fk_classificacao']),
                int(row['fk_evolucao']),
                int(row['fk_sintomas']),
                int(row['vl_caso_confirmado']),
                int(row['vl_caso_grave']),
                int(row['vl_obito']),
                int(row['vl_hospitalizado']),
                int(row['vl_qtd_sintomas']),
                int(row['vl_qtd_alarmes']),
                float(row['vl_idade_anos']) if pd.notna(row['vl_idade_anos']) else None,
                row['ts_notificacao'],
                row['ts_sintomas'] if pd.notna(row['ts_sintomas']) else None
            ))
        
        # Executar batch insert
        cursor.executemany(insert_sql, batch_data)
        conn.commit()
        
        progress = (batch_num + 1) / total_batches * 100
        print(f"   Lote {batch_num + 1}/{total_batches} ({progress:.1f}%) - {end_idx:,} registros")
    
    print(f"\n✅ Carga da tabela fato concluída: {total_rows:,} registros")
    
except Exception as e:
    conn.rollback()
    print(f"❌ Erro na carga da fato: {e}")
    raise
finally:
    cursor.close()
    conn.close()

Limpando tabela fato...
Inserindo 1,661,634 registros em 34 lotes...
   Lote 1/34 (2.9%) - 50,000 registros
   Lote 2/34 (5.9%) - 100,000 registros
   Lote 3/34 (8.8%) - 150,000 registros
   Lote 4/34 (11.8%) - 200,000 registros
   Lote 5/34 (14.7%) - 250,000 registros
   Lote 6/34 (17.6%) - 300,000 registros
   Lote 7/34 (20.6%) - 350,000 registros
   Lote 8/34 (23.5%) - 400,000 registros
   Lote 9/34 (26.5%) - 450,000 registros
   Lote 10/34 (29.4%) - 500,000 registros
   Lote 11/34 (32.4%) - 550,000 registros
   Lote 12/34 (35.3%) - 600,000 registros
   Lote 13/34 (38.2%) - 650,000 registros
   Lote 14/34 (41.2%) - 700,000 registros
   Lote 15/34 (44.1%) - 750,000 registros
   Lote 16/34 (47.1%) - 800,000 registros
   Lote 17/34 (50.0%) - 850,000 registros
   Lote 18/34 (52.9%) - 900,000 registros
   Lote 19/34 (55.9%) - 950,000 registros
   Lote 20/34 (58.8%) - 1,000,000 registros
   Lote 21/34 (61.8%) - 1,050,000 registros
   Lote 22/34 (64.7%) - 1,100,000 registros
   Lote 23/34 

## 8. Validações de Qualidade

In [25]:
# Validações conforme documento - Seção 8
print("="*60)
print("VALIDAÇÕES DE QUALIDADE - GOLD LAYER")
print("="*60)

conn = get_connection()

# 8.1 Validações de Integridade
print("\n📋 8.1 VALIDAÇÕES DE INTEGRIDADE")

# Contagem Silver vs Gold
count_silver = pd.read_sql("SELECT COUNT(*) as cnt FROM public.dengue_silver", conn)['cnt'][0]
count_gold = pd.read_sql("SELECT COUNT(*) as cnt FROM gold.ft_dengue", conn)['cnt'][0]

print(f"   ✅ Registros Silver: {count_silver:,}")
print(f"   ✅ Registros Gold (ft_dengue): {count_gold:,}")
print(f"   {'✅' if count_silver == count_gold else '❌'} Volumetria: {'VÁLIDA' if count_silver == count_gold else 'INVÁLIDA'}")

VALIDAÇÕES DE QUALIDADE - GOLD LAYER

📋 8.1 VALIDAÇÕES DE INTEGRIDADE
   ✅ Registros Silver: 1,661,634
   ✅ Registros Gold (ft_dengue): 1,661,634
   ✅ Volumetria: VÁLIDA


In [26]:
# Verificar FKs órfãs
print("\n   Verificação de integridade referencial:")

# Verificar cada FK
fk_checks = [
    ('fk_tempo', 'dim_tempo', 'sk_tempo'),
    ('fk_localizacao', 'dim_localizacao', 'sk_localizacao'),
    ('fk_paciente', 'dim_paciente', 'sk_paciente'),
    ('fk_classificacao', 'dim_classificacao', 'sk_classificacao'),
    ('fk_evolucao', 'dim_evolucao', 'sk_evolucao'),
    ('fk_sintomas', 'dim_sintomas', 'sk_sintomas')
]

for fk_col, dim_table, sk_col in fk_checks:
    query = f"""
        SELECT COUNT(*) as orphans 
        FROM gold.ft_dengue f
        LEFT JOIN gold.{dim_table} d ON f.{fk_col} = d.{sk_col}
        WHERE d.{sk_col} IS NULL
    """
    orphans = pd.read_sql(query, conn)['orphans'][0]
    status = '✅' if orphans == 0 else '❌'
    print(f"   {status} {fk_col} → {dim_table}: {orphans} órfãos")


   Verificação de integridade referencial:
   ✅ fk_tempo → dim_tempo: 0 órfãos
   ✅ fk_localizacao → dim_localizacao: 0 órfãos
   ✅ fk_paciente → dim_paciente: 0 órfãos
   ✅ fk_classificacao → dim_classificacao: 0 órfãos
   ✅ fk_evolucao → dim_evolucao: 0 órfãos
   ✅ fk_sintomas → dim_sintomas: 0 órfãos


In [27]:
# 8.2 Validações de Negócio (conforme documento)
print("\n📋 8.2 VALIDAÇÕES DE NEGÓCIO")

# Valores esperados do documento
EXPECTED = {
    'casos_confirmados': 1445765,
    'casos_graves': 37208,
    'obitos': 1773,
    'hospitalizacoes': 72684
}

# Buscar valores reais
metricas = pd.read_sql("""
    SELECT 
        SUM(vl_caso_confirmado) as casos_confirmados,
        SUM(vl_caso_grave) as casos_graves,
        SUM(vl_obito) as obitos,
        SUM(vl_hospitalizado) as hospitalizacoes
    FROM gold.ft_dengue
""", conn)

for metrica, esperado in EXPECTED.items():
    real = int(metricas[metrica][0])
    status = '✅' if real == esperado else '⚠️'
    diff = real - esperado
    print(f"   {status} {metrica}: {real:,} (esperado: {esperado:,}, diff: {diff:+,})")


📋 8.2 VALIDAÇÕES DE NEGÓCIO
   ✅ casos_confirmados: 1,445,765 (esperado: 1,445,765, diff: +0)
   ✅ casos_graves: 37,208 (esperado: 37,208, diff: +0)
   ✅ obitos: 1,773 (esperado: 1,773, diff: +0)
   ✅ hospitalizacoes: 72,684 (esperado: 72,684, diff: +0)


In [28]:
# Taxa de letalidade
letalidade = pd.read_sql("""
    SELECT 
        ROUND(100.0 * SUM(vl_obito) / SUM(vl_caso_confirmado), 3) as taxa_letalidade
    FROM gold.ft_dengue
    WHERE vl_caso_confirmado = 1
""", conn)

taxa = letalidade['taxa_letalidade'][0]
print(f"   ✅ Taxa de letalidade: {taxa}% (esperado: ~0.1%)")

   ✅ Taxa de letalidade: 0.123% (esperado: ~0.1%)


In [29]:
# Período dos dados
periodo = pd.read_sql("""
    SELECT 
        MIN(ts_notificacao) as data_min,
        MAX(ts_notificacao) as data_max
    FROM gold.ft_dengue
""", conn)

print(f"   ✅ Período: {periodo['data_min'][0]} a {periodo['data_max'][0]}")
print(f"      (esperado: 29/12/2024 a 05/01/2026)")

   ✅ Período: 2024-12-29 a 2026-01-05
      (esperado: 29/12/2024 a 05/01/2026)


In [30]:
# Distribuição por UF (top 5)
print("\n   Top 5 UFs por casos:")
top_ufs = pd.read_sql("""
    SELECT 
        l.uf_sigla,
        SUM(f.vl_caso_confirmado) as casos
    FROM gold.ft_dengue f
    JOIN gold.dim_localizacao l ON f.fk_localizacao = l.sk_localizacao
    GROUP BY l.uf_sigla
    ORDER BY casos DESC
    LIMIT 5
""", conn)

for _, row in top_ufs.iterrows():
    print(f"      {row['uf_sigla']}: {int(row['casos']):,} casos")

print(f"      (esperado: SP líder com 876.832 casos)")


   Top 5 UFs por casos:
      SP: 876,832 casos
      MG: 119,016 casos
      GO: 96,685 casos
      PR: 92,514 casos
      RS: 44,075 casos
      (esperado: SP líder com 876.832 casos)


In [31]:
# 8.3 Validações de Qualidade
print("\n📋 8.3 VALIDAÇÕES DE QUALIDADE")

# FKs obrigatórias não nulas
fks_null = pd.read_sql("""
    SELECT 
        SUM(CASE WHEN fk_tempo IS NULL THEN 1 ELSE 0 END) as tempo_null,
        SUM(CASE WHEN fk_localizacao IS NULL THEN 1 ELSE 0 END) as loc_null,
        SUM(CASE WHEN fk_paciente IS NULL THEN 1 ELSE 0 END) as pac_null,
        SUM(CASE WHEN fk_classificacao IS NULL THEN 1 ELSE 0 END) as class_null,
        SUM(CASE WHEN fk_evolucao IS NULL THEN 1 ELSE 0 END) as evol_null,
        SUM(CASE WHEN fk_sintomas IS NULL THEN 1 ELSE 0 END) as sint_null
    FROM gold.ft_dengue
""", conn)

total_nulls = sum(fks_null.iloc[0])
print(f"   {'✅' if total_nulls == 0 else '❌'} FKs nulas: {total_nulls}")

# Flags booleanos apenas 0 ou 1
flags_invalid = pd.read_sql("""
    SELECT COUNT(*) as invalid
    FROM gold.ft_dengue
    WHERE vl_caso_confirmado NOT IN (0,1)
       OR vl_caso_grave NOT IN (0,1)
       OR vl_obito NOT IN (0,1)
       OR vl_hospitalizado NOT IN (0,1)
""", conn)['invalid'][0]

print(f"   {'✅' if flags_invalid == 0 else '❌'} Flags inválidos (fora 0/1): {flags_invalid}")

# Idades válidas (0-120 ou NULL)
idades_invalid = pd.read_sql("""
    SELECT COUNT(*) as invalid
    FROM gold.ft_dengue
    WHERE vl_idade_anos IS NOT NULL 
      AND (vl_idade_anos < 0 OR vl_idade_anos > 120)
""", conn)['invalid'][0]

print(f"   {'✅' if idades_invalid == 0 else '❌'} Idades inválidas (fora 0-120): {idades_invalid}")

conn.close()


📋 8.3 VALIDAÇÕES DE QUALIDADE
   ✅ FKs nulas: 0
   ✅ Flags inválidos (fora 0/1): 0
   ✅ Idades inválidas (fora 0-120): 0


## 9. Métricas de Qualidade

In [32]:
# Métricas conforme documento - Seção 9
print("\n" + "="*60)
print("MÉTRICAS DE QUALIDADE - GOLD LAYER")
print("="*60)

conn = get_connection()

# Completude (campos obrigatórios preenchidos)
completude = pd.read_sql("""
    SELECT 
        ROUND(100.0 * SUM(CASE WHEN fk_tempo > 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as tempo,
        ROUND(100.0 * SUM(CASE WHEN fk_localizacao > 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as localizacao,
        ROUND(100.0 * SUM(CASE WHEN fk_paciente > 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as paciente,
        ROUND(100.0 * SUM(CASE WHEN fk_classificacao > 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as classificacao,
        ROUND(100.0 * SUM(CASE WHEN fk_evolucao > 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as evolucao,
        ROUND(100.0 * SUM(CASE WHEN fk_sintomas > 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as sintomas
    FROM gold.ft_dengue
""", conn)

avg_completude = completude.iloc[0].mean()
print(f"\n📊 COMPLETUDE (esperado: 95%+)")
print(f"   Média geral: {avg_completude:.2f}%")
for col in completude.columns:
    print(f"   - {col}: {completude[col][0]}%")

# Consistência (FKs válidas)
print(f"\n📊 CONSISTÊNCIA (esperado: 100%)")
print(f"   Integridade referencial: 100% (verificado acima)")

# Acurácia (regras de negócio)
print(f"\n📊 ACURÁCIA (esperado: 99%+)")
print(f"   Regras de negócio: VALIDADAS (verificado acima)")

conn.close()

print("\n" + "="*60)
print("✅ ETL SILVER TO GOLD CONCLUÍDO COM SUCESSO!")
print("="*60)


MÉTRICAS DE QUALIDADE - GOLD LAYER

📊 COMPLETUDE (esperado: 95%+)
   Média geral: 100.00%
   - tempo: 100.0%
   - localizacao: 100.0%
   - paciente: 100.0%
   - classificacao: 100.0%
   - evolucao: 100.0%
   - sintomas: 100.0%

📊 CONSISTÊNCIA (esperado: 100%)
   Integridade referencial: 100% (verificado acima)

📊 ACURÁCIA (esperado: 99%+)
   Regras de negócio: VALIDADAS (verificado acima)

✅ ETL SILVER TO GOLD CONCLUÍDO COM SUCESSO!


## 10. Queries de Exemplo (BI)

In [33]:
# Queries típicas de BI conforme documento - Seção 10
conn = get_connection()

print("📊 QUERY 1: Top 5 UFs com maior taxa de letalidade")
print("-" * 50)

query1 = """
SELECT 
    l.uf_sigla,
    SUM(f.vl_caso_confirmado) as casos,
    SUM(f.vl_obito) as obitos,
    ROUND(100.0 * SUM(f.vl_obito) / NULLIF(SUM(f.vl_caso_confirmado), 0), 3) as taxa_letalidade
FROM gold.ft_dengue f
JOIN gold.dim_localizacao l ON f.fk_localizacao = l.sk_localizacao
GROUP BY l.uf_sigla
HAVING SUM(f.vl_caso_confirmado) > 1000
ORDER BY taxa_letalidade DESC
LIMIT 5
"""

pd.read_sql(query1, conn)

📊 QUERY 1: Top 5 UFs com maior taxa de letalidade
--------------------------------------------------


,uf_sigla,casos,obitos,taxa_letalidade
0,MS,8962,21,0.234
1,PA,14182,32,0.226
2,RN,4228,7,0.166
3,PR,92514,145,0.157
4,PI,7449,11,0.148


In [34]:
print("📊 QUERY 2: Evolução semanal de casos graves")
print("-" * 50)

query2 = """
SELECT 
    t.ano,
    t.semana_epi,
    SUM(f.vl_caso_grave) as casos_graves
FROM gold.ft_dengue f
JOIN gold.dim_tempo t ON f.fk_tempo = t.sk_tempo
GROUP BY t.ano, t.semana_epi
ORDER BY t.ano, t.semana_epi
LIMIT 20
"""

pd.read_sql(query2, conn)

📊 QUERY 2: Evolução semanal de casos graves
--------------------------------------------------


,ano,semana_epi,casos_graves
0,2024,1,34
1,2024,52,2
2,2025,1,263
3,2025,2,701
4,2025,3,886
5,2025,4,1078
6,2025,5,1232
7,2025,6,1264
8,2025,7,1337
9,2025,8,1469


In [36]:
print("📊 QUERY 3: Perfil demográfico mais afetado")
print("-" * 50)

query3 = """
SELECT 
    p.faixa_etaria,
    p.sexo_desc,
    COUNT(*) as total_casos,
    SUM(f.vl_caso_confirmado) as casos_confirmados,
    ROUND(AVG(f.vl_idade_anos)::numeric, 1) as idade_media
FROM gold.ft_dengue f
JOIN gold.dim_paciente p ON f.fk_paciente = p.sk_paciente
GROUP BY p.faixa_etaria, p.sexo_desc
ORDER BY casos_confirmados DESC
LIMIT 10
"""

pd.read_sql(query3, conn)

📊 QUERY 3: Perfil demográfico mais afetado
--------------------------------------------------


,faixa_etaria,sexo_desc,total_casos,casos_confirmados,idade_media
0,20-39 anos,Feminino,319843,274735,29.3
1,40-59 anos,Feminino,257793,229903,48.7
2,20-39 anos,Masculino,267704,229568,29.1
3,40-59 anos,Masculino,191312,170703,48.5
4,60+ anos,Feminino,137272,125163,69.5
5,10-19 anos,Masculino,121508,103861,14.8
6,10-19 anos,Feminino,116904,99151,15.1
7,60+ anos,Masculino,99518,90304,69.5
8,5-9 anos,Masculino,47327,39608,7.1
9,5-9 anos,Feminino,41265,34744,7.2


In [37]:
conn.close()
print("\n✅ Demonstração de queries BI concluída!")


✅ Demonstração de queries BI concluída!
